**PREGUNTAS CASO NEGOCIO SPARK**

Para la resolución de los ejercicios del Caso de Negocio, se partirá de las tablas iniciales que se han guardado en Hive al comienzo de la práctica resuelta

In [0]:
# Se realizan todos los imports necesarios
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window


En primer lugar, creamos las tablas sobre las que lanzaremos las consultas y transformaciones, y que hemos extraído del ejercicio de repaso.

En este caso, al trabajar directamente sobre Databricks, omitiremos en los comandos la instrucción de guardar las tablas en la base de datos "sector_telecomunicaciones", y trabajaremos directamente con la base de datos por defecto.

In [0]:
columns = ['id_cliente','nombre','edad','sexo','provincia','pais']
data = [(1,'Pablo Perez',26,'M','Madrid','España'),(2,'Eduardo Redondo',58,'M','Bogota','Colombia'),(3,'Roberto Salazar',68,'M','Monterrey','Mexico'),
        (4,'Pedro Conde',32,'M','Madrid','España'),(5,'Ana Robles',41,'F','Valladolid','España'),(6,'David Roldan',74,'M','Guadalajara','Mexico'),
        (7,'Carmen Arauzo',19,'F','Medellin','Colombia'),(8,'Silvia Rodriguez',49,'F','Barcelona','España'),(9,'David Cardoso',24,'M','Cali','Colombia'),
        (10,'Pablo Lopez',40,'M','Sevilla','España'),(11,'Luis Rodrigues',20,'M','Bogota','Colombia'),(12,'Alvaro Monroy',75,'M','Monterrey','Mexico'),
        (13,'Victor Ruiz',53,'M','Valladolid','España'),(14,'Melisa Aguado',30,'F','Bogota','Colombia'),(15,'Cristian Cuadrado',52,'M','Guadalajara','Mexico'),
        (16,'Cristina Sanz',49,'F','Madrid','España'),(17,'Jorge Recio',18,'M','Valladolid','España'),(18,'Laura Luiz',44,'F','Cali','Colombia'),
        (19,'Juan Carlos Iglesias',38,'M','Barcelona','España'),(20,'Oscar Rico',22,'M','Valladolid','España'),(21,'Fatima Cuevas',29,'F','Cali','Colombia'),
        (22,'Clara Suarez',21,'F','Sevilla','España'),(23,'Fernanda Gomez',78,'F','Monterrey','Mexico'),(24,'Ruben Garcia',68,'M','Sevilla','España'),
        (25,'Ines Barcero',29,'F','Guadalajara','Mexico'),(26,'Celia Castro',47,'F','Barcelona','España'),(27,'Roberta Varado',64,'F','Cali','Colombia'),
        (28,'Walter Ramos',54,'M','Cali','Colombia'),(29,'Romina Verde',29,'F','Guadalajara','Mexico'),(30,'Marta Rodrigo',67,'F','Guadalajara','Mexico')]

df_clientes = spark.createDataFrame(data, columns)
spark.conf.set("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation","true")
# Si nos da error el modo overwrite, descomentar el código de la anterior línea. 
# Este fragmento nos permite evitar errores al sobre escribir la tabla (mode("overwrite")) si el trabajo se ha interrumpido previamente mientras había una operación de escritura en curso.
# https://docs.microsoft.com/es-es/azure/databricks/kb/jobs/spark-overwrite-cancel/
df_clientes.write.mode("overwrite").saveAsTable('df_clientes')


columns = ['id_oferta','descripcion','importe'] 
data = [(1,'Fibra Optica 300MB + Fijo ilimitado + Television',70.00),
        (2,'Fibra Optica 600MB + Fijo ilimitado + Television', 78.50),
        (3,'Fibra Optica 300MB + Movil llamadas y datos ilimitados + Television', 85.00),
        (4,'Fibra Optica 600MB + Movil llamadas y datos ilimitados + Television', 95.00),
        (5,'Fibra Optica 300MB + Fijo ilimitado + Movil llamadas y datos ilimitados', 102.50),
        (6,'Fibra Optica 600MB + Fijo ilimitado + Movil llamadas y datos ilimitados + Television', 124.50),
        (7,'Fibra Optica 300MB + Movil llamadas ilimitadas + Television', 80.50),
        (8,'Fibra Optica 300MB + Movil datos ilimitados + Television', 84.00),
        (9,'Fijo llamadas ilimitadas + Movil llamadas y datos ilimitados', 42.50),
        (10,'Fijo llamadas ilimitadas + Movil llamadas ilimitadas', 31.99)]

df_ofertas = spark.createDataFrame(data, columns)
df_ofertas.write.mode("overwrite").saveAsTable('df_ofertas')

columns = ['id_cliente','id_oferta','importe','fecha'] 
data = [(1,6,124.5,"2020-07-31"),(1,6,118.60,"2020-07-31"),(1,9,36.5,"2020-07-31"),
        (2,4,95.0,"2020-07-31"),(3,5,102.5,"2020-07-31"),(5,7,80.5,"2020-07-31"),
        (5,9,40.5,"2020-07-31"),(6,8,84.0,"2020-07-31"),(7,4,95.0,"2020-07-31"),
        (8,5,102.5,"2020-07-31"),(10,6,124.5,"2020-07-31"),(11,3,80.0,"2020-07-31"),
        (12,1,70.0,"2020-07-31"),(13,4,95.0,"2020-07-31"),(14,9,42.5,"2020-07-31"),
        (15,2,78.50,"2020-07-31"),(15,6,115.0,"2020-07-31"),(16,1,70.0,"2020-07-31"),
        (17,8,84.0,"2020-07-31"),(18,5,102.5,"2020-07-31"),(18,9,37.5,"2020-07-31"),
        (20,4,95.0,"2020-07-31"),(22,2,78.5,"2020-07-31"),(22,7,69.5,"2020-07-31"),
        (23,3,85.0,"2020-07-31"),(24,3,85.0,"2020-07-31"),(25,9,42.5,"2020-07-31"),
        (25,10,31.99,"2020-07-31"),(26,4,95.0,"2020-07-31"),(27,8,84.0,"2020-07-31"),
        (27,9,41.5,"2020-07-31"),(29,6,124.5,"2020-07-31"),(30,1,70.0,"2020-07-31"),
        (30,2,70.0,"2020-07-31")]
        
df_facturas_mes_ant = spark.createDataFrame(data, columns)
df_facturas_mes_ant.write.mode("overwrite").saveAsTable('df_facturas_mes_ant')

columns = ['id_cliente','id_oferta','importe','fecha'] 
data = [(1,6,124.5,"2020-08-31"),(1,6,118.60,"2020-08-31"),(2,4,95.0,"2020-08-31"),
        (3,5,102.5,"2020-08-31"),(4,1,70.0,"2020-08-31"),(5,7,80.5,"2020-08-31"),
        (5,9,40.5,"2020-08-31"),(6,8,84.0,"2020-08-31"),(7,4,95.0,"2020-08-31"),
        (8,5,100.5,"2020-08-31"),(9,7,78.50,"2020-08-31"),(9,10,29.99,"2020-08-31"),
        (10,6,124.5,"2020-08-31"),(11,3,83.0,"2020-08-31"),(12,1,70.0,"2020-08-31"),
        (13,4,95.0,"2020-08-31"),(14,9,42.5,"2020-08-31"),(15,2,78.50,"2020-08-31"),
        (15,6,115.0,"2020-08-31"),(17,8,84.0,"2020-08-31"),(18,5,102.5,"2020-08-31"),
        (18,9,39.0,"2020-08-31"),(19,3,85.0,"2020-08-31"),(21,6,124.50,"2020-08-31"),
        (22,2,78.5,"2020-08-31"),(22,7,72.5,"2020-08-31"),(23,3,85.0,"2020-08-31"),
        (25,9,42.5,"2020-08-31"),(25,10,29.70,"2020-08-31"),(26,4,95.0,"2020-08-31"),
        (27,8,84.0,"2020-08-31"),(28,5,102.50,"2020-08-31"),(29,6,121.50,"2020-08-31"),
        (30,1,70.0,"2020-08-31"),(30,2,71.0,"2020-08-31")]

df_facturas_mes_actual = spark.createDataFrame(data, columns)
df_facturas_mes_actual.write.mode("overwrite").saveAsTable('df_facturas_mes_actual')

columns = ['id_cliente','consumo_datos_MB','sms_enviados','minutos_llamadas_movil','minutos_llamadas_fijo','fecha']
data = [(1,664,3,25,0,"2020-08-01"),(1,943,0,12,5,"2020-08-02"),(1,1035,1,46,10,"2020-08-03"),(1,760,3,17,0,"2020-08-04"),
        (1,1409,1,31,4,"2020-08-05"),(1,1627,0,20,0,"2020-08-06"),(1,739,3,21,0,"2020-08-07"),(1,760,0,12,0,"2020-08-08"),
        (1,928,0,0,24,"2020-08-09"),(1,829,2,46,0,"2020-08-10"),(1,1810,3,0,0,"2020-08-11"),(1,760,0,25,0,"2020-08-12"),
        (1,603,0,41,0,"2020-08-13"),(1,732,0,34,102,"2020-08-14"),(1,729,1,0,50,"2020-08-15"),(1,760,1,31,0,"2020-08-16"),
        (1,1934,2,78,0,"2020-08-17"),(1,827,0,29,32,"2020-08-18"),(1,691,0,0,7,"2020-08-19"),(1,760,2,19,0,"2020-08-20"),
        (1,1142,5,14,0,"2020-08-21"),(1,1242,1,40,0,"2020-08-22"),(1,830,0,32,2,"2020-08-23"),(1,760,4,56,17,"2020-08-24"),
        (1,734,0,19,0,"2020-08-25"),(1,1729,0,11,0,"2020-08-26"),(1,819,1,95,18,"2020-08-27"),(1,760,0,60,12,"2020-08-28"),
        (1,302,0,29,0,"2020-08-29"),(1,497,3,14,0,"2020-08-30"),(1,1281,2,21,0,"2020-08-31"),
        (2,321,7,38,13,"2020-08-01"),(2,145,0,8,0,"2020-08-02"),(2,420,1,46,4,"2020-08-03"),(2,492,1,9,0,"2020-08-04"),
        (2,264,8,29,31,"2020-08-05"),(2,172,10,22,0,"2020-08-06"),(2,314,2,74,34,"2020-08-07"),(2,312,2,0,16,"2020-08-08"),
        (2,312,11,32,27,"2020-08-09"),(2,243,2,48,0,"2020-08-10"),(2,423,4,38,12,"2020-08-11"),(2,492,1,10,22,"2020-08-12"),
        (2,492,12,29,29,"2020-08-13"),(2,274,4,60,0,"2020-08-14"),(2,402,0,28,20,"2020-08-15"),(2,471,0,12,30,"2020-08-16"),
        (2,462,1,36,32,"2020-08-17"),(2,294,3,78,42,"2020-08-18"),(2,593,0,29,65,"2020-08-19"),(2,380,12,53,9,"2020-08-20"),
        (2,568,2,42,20,"2020-08-21"),(2,300,2,40,35,"2020-08-22"),(2,600,0,35,29,"2020-08-23"),(2,397,9,37,40,"2020-08-24"),
        (2,623,4,59,42,"2020-08-25"),(2,302,1,34,15,"2020-08-26"),(2,319,1,52,34,"2020-08-27"),(2,901,8,28,83,"2020-08-28"),
        (2,611,0,41,38,"2020-08-29"),(2,319,0,21,19,"2020-08-30"),(2,387,3,19,18,"2020-08-31"),
        (3,123,5,42,14,"2020-08-01"),(3,460,4,24,16,"2020-08-02"),(3,291,3,27,18,"2020-08-03"),(3,149,7,20,0,"2020-08-04"),
        (3,182,6,20,19,"2020-08-05"),(3,245,12,28,21,"2020-08-06"),(3,302,2,31,26,"2020-08-07"),(3,200,8,18,8,"2020-08-08"),
        (3,239,7,40,25,"2020-08-09"),(3,111,9,29,0,"2020-08-10"),(3,191,1,14,29,"2020-08-11"),(3,28,12,4,10,"2020-08-12"),
        (3,201,3,15,32,"2020-08-13"),(3,102,8,31,0,"2020-08-14"),(3,62,10,29,31,"2020-08-15"),(3,82,2,49,42,"2020-08-16"),
        (3,72,1,0,15,"2020-08-17"),(3,94,2,12,18,"2020-08-18"),(3,30,11,20,45,"2020-08-19"),(3,172,1,63,8,"2020-08-20"),
        (3,80,1,12,30,"2020-08-21"),(3,274,7,5,11,"2020-08-22"),(3,219,7,8,15,"2020-08-23"),(3,104,7,6,2,"2020-08-24"),
        (3,180,9,48,45,"2020-08-25"),(3,210,0,10,44,"2020-08-26"),(3,310,2,17,18,"2020-08-27"),(3,182,9,0,1,"2020-08-28"),
        (3,164,0,32,5,"2020-08-29"),(3,289,0,12,33,"2020-08-30"),(3,126,3,23,22,"2020-08-31"),
        (4,523,0,87,0,"2020-08-01"),(4,628,1,56,0,"2020-08-02"),(4,1021,0,15,0,"2020-08-03"),(4,610,1,45,0,"2020-08-04"),
        (4,612,0,65,0,"2020-08-05"),(4,359,0,102,15,"2020-08-06"),(4,832,1,62,0,"2020-08-07"),(4,583,0,30,0,"2020-08-08"),
        (4,490,2,81,20,"2020-08-09"),(4,479,0,81,0,"2020-08-10"),(4,792,2,19,0,"2020-08-11"),(4,1216,0,27,32,"2020-08-12"),
        (4,512,0,32,0,"2020-08-13"),(4,824,4,76,0,"2020-08-14"),(4,682,1,37,18,"2020-08-15"),(4,829,2,47,0,"2020-08-16"),
        (4,921,0,45,0,"2020-08-17"),(4,729,0,42,0,"2020-08-18"),(4,534,2,95,64,"2020-08-19"),(4,967,3,51,82,"2020-08-20"),
        (4,721,3,54,0,"2020-08-21"),(4,218,0,38,17,"2020-08-22"),(4,729,0,85,0,"2020-08-23"),(4,826,1,38,0,"2020-08-24"),
        (4,845,1,34,0,"2020-08-25"),(4,367,2,71,21,"2020-08-26"),(4,502,0,60,0,"2020-08-27"),(4,800,0,25,0,"2020-08-28"),
        (4,629,0,49,0,"2020-08-29"),(4,588,0,50,0,"2020-08-30"),(4,693,1,31,40,"2020-08-31"),
        (5,241,1,125,20,"2020-08-01"),(5,315,0,36,32,"2020-08-02"),(5,301,1,20,80,"2020-08-03"),(5,152,5,38,62,"2020-08-04"),
        (5,289,4,110,45,"2020-08-05"),(5,510,2,74,40,"2020-08-06"),(5,178,2,19,67,"2020-08-07"),(5,241,3,41,59,"2020-08-08"),
        (5,301,6,62,39,"2020-08-09"),(5,86,4,82,47,"2020-08-10"),(5,201,6,75,61,"2020-08-11"),(5,358,8,51,62,"2020-08-12"),
        (5,176,8,45,0,"2020-08-13"),(5,109,5,102,12,"2020-08-14"),(5,798,5,91,45,"2020-08-15"),(5,301,0,43,71,"2020-08-16"),
        (5,198,9,73,70,"2020-08-17"),(5,125,2,115,17,"2020-08-18"),(5,182,4,99,49,"2020-08-19"),(5,222,0,30,80,"2020-08-20"),
        (5,234,3,82,34,"2020-08-21"),(5,201,3,70,20,"2020-08-22"),(5,283,3,130,27,"2020-08-23"),(5,333,2,28,0,"2020-08-24"),
        (5,387,5,90,41,"2020-08-25"),(5,258,5,62,30,"2020-08-26"),(5,251,7,100,36,"2020-08-27"),(5,475,6,79,16,"2020-08-28"),
        (5,312,12,15,80,"2020-08-29"),(5,328,1,50,19,"2020-08-30"),(5,291,11,32,65,"2020-08-31"),
        (6,50,7,10,50,"2020-08-01"),(6,80,6,10,0,"2020-08-02"),(6,65,12,0,40,"2020-08-03"),(6,284,14,0,42,"2020-08-04"),
        (6,20,9,0,46,"2020-08-05"),(6,36,4,0,67,"2020-08-06"),(6,90,9,10,36,"2020-08-07"),(6,201,18,0,83,"2020-08-08"),
        (6,28,13,0,64,"2020-08-09"),(6,150,7,22,46,"2020-08-10"),(6,135,7,0,65,"2020-08-11"),(6,196,22,17,68,"2020-08-12"),
        (6,49,15,0,37,"2020-08-13"),(6,0,0,0,92,"2020-08-14"),(6,122,6,0,56,"2020-08-15"),(6,146,3,0,94,"2020-08-16"),
        (6,102,13,15,48,"2020-08-17"),(6,38,8,0,17,"2020-08-18"),(6,180,11,14,41,"2020-08-19"),(6,159,6,0,26,"2020-08-20"),
        (6,27,16,24,50,"2020-08-21"),(6,156,12,0,75,"2020-08-22"),(6,193,10,20,39,"2020-08-23"),(6,478,4,30,64,"2020-08-24"),
        (6,35,5,0,80,"2020-08-25"),(6,120,15,28,52,"2020-08-26"),(6,201,8,0,54,"2020-08-27"),(6,310,1,28,15,"2020-08-28"),
        (6,40,3,8,35,"2020-08-29"),(6,73,4,16,34,"2020-08-30"),(6,174,7,25,13,"2020-08-31"),
        (7,1456,0,37,0,"2020-08-01"),(7,946,0,24,24,"2020-08-02"),(7,1302,0,72,0,"2020-08-03"),(7,1356,0,13,45,"2020-08-04"),
        (7,1256,1,28,0,"2020-08-05"),(7,985,0,11,5,"2020-08-06"),(7,934,0,40,0,"2020-08-07"),(7,464,1,65,0,"2020-08-08"),
        (7,930,0,0,0,"2020-08-09"),(7,1385,0,30,0,"2020-08-10"),(7,635,0,38,0,"2020-08-11"),(7,588,2,60,0,"2020-08-12"),
        (7,849,2,40,0,"2020-08-13"),(7,1405,3,10,0,"2020-08-14"),(7,911,2,67,24,"2020-08-15"),(7,1495,0,83,0,"2020-08-16"),
        (7,830,0,47,23,"2020-08-17"),(7,1295,0,52,0,"2020-08-18"),(7,844,1,9,0,"2020-08-19"),(7,1246,0,12,4,"2020-08-20"),
        (7,1044,0,28,0,"2020-08-21"),(7,732,1,0,0,"2020-08-22"),(7,1468,4,13,0,"2020-08-23"),(7,967,0,79,0,"2020-08-24"),
        (7,1953,1,17,19,"2020-08-25"),(7,802,0,24,11,"2020-08-26"),(7,1853,1,71,15,"2020-08-27"),(7,1344,0,61,16,"2020-08-28"),
        (7,1067,2,13,0,"2020-08-29"),(7,1740,0,33,57,"2020-08-30"),(7,1122,0,38,12,"2020-08-31"),
        (8,346,5,75,30,"2020-08-01"),(8,502,6,74,40,"2020-08-02"),(8,482,1,12,62,"2020-08-03"),(8,200,10,63,69,"2020-08-04"),
        (8,386,6,83,29,"2020-08-05"),(8,310,3,62,28,"2020-08-06"),(8,500,3,123,0,"2020-08-07"),(8,284,5,71,31,"2020-08-08"),
        (8,305,12,42,19,"2020-08-09"),(8,375,13,80,15,"2020-08-10"),(8,319,5,84,29,"2020-08-11"),(8,249,4,42,53,"2020-08-12"),
        (8,401,10,26,0,"2020-08-13"),(8,281,15,37,76,"2020-08-14"),(8,318,7,86,34,"2020-08-15"),(8,394,8,39,12,"2020-08-16"),
        (8,250,8,56,63,"2020-08-17"),(8,272,4,42,59,"2020-08-18"),(8,492,1,70,46,"2020-08-19"),(8,365,3,51,33,"2020-08-20"),
        (8,284,2,12,25,"2020-08-21"),(8,266,5,50,15,"2020-08-22"),(8,172,1,67,12,"2020-08-23"),(8,402,0,68,8,"2020-08-24"),
        (8,265,4,103,31,"2020-08-25"),(8,388,9,62,43,"2020-08-26"),(8,193,1,48,43,"2020-08-27"),(8,95,6,77,18,"2020-08-28"),
        (8,192,0,93,22,"2020-08-29"),(8,293,12,71,55,"2020-08-30"),(8,174,1,52,36,"2020-08-31"),
        (9,1304,0,13,0,"2020-08-01"),(9,720,0,11,5,"2020-08-02"),(9,1111,1,8,12,"2020-08-03"),(9,1760,1,5,1,"2020-08-04"),
        (9,1502,1,0,3,"2020-08-05"),(9,1455,2,21,0,"2020-08-06"),(9,850,2,7,0,"2020-08-07"),(9,799,2,7,7,"2020-08-08"),
        (9,623,0,5,0,"2020-08-09"),(9,1294,1,5,0,"2020-08-10"),(9,950,0,4,0,"2020-08-11"),(9,923,0,15,0,"2020-08-12"),
        (9,730,0,8,0,"2020-08-13"),(9,823,0,0,0,"2020-08-14"),(9,957,0,0,7,"2020-08-15"),(9,829,0,0,2,"2020-08-16"),
        (9,945,0,0,5,"2020-08-17"),(9,392,0,0,3,"2020-08-18"),(9,1028,0,3,12,"2020-08-19"),(9,729,0,0,0,"2020-08-20"),
        (9,920,4,4,0,"2020-08-21"),(9,720,1,2,9,"2020-08-22"),(9,1932,0,8,27,"2020-08-23"),(9,999,3,0,6,"2020-08-24"),
        (9,869,0,8,0,"2020-08-25"),(9,1830,0,20,15,"2020-08-26"),(9,1234,1,0,0,"2020-08-27"),(9,1885,4,24,13,"2020-08-28"),
        (9,888,2,13,0,"2020-08-29"),(9,929,1,7,6,"2020-08-30"),(9,606,2,37,0,"2020-08-31"),
        (10,482,3,35,20,"2020-08-01"),(10,418,0,32,18,"2020-08-02"),(10,382,3,22,24,"2020-08-03"),(10,212,2,30,16,"2020-08-04"),
        (10,510,7,43,6,"2020-08-05"),(10,399,1,43,25,"2020-08-06"),(10,593,0,23,25,"2020-08-07"),(10,392,2,4,25,"2020-08-08"),
        (10,692,0,13,17,"2020-08-09"),(10,312,0,2,15,"2020-08-10"),(10,492,0,13,27,"2020-08-11"),(10,433,0,32,18,"2020-08-12"),
        (10,582,2,0,29,"2020-08-13"),(10,422,3,31,6,"2020-08-14"),(10,598,1,4,46,"2020-08-15"),(10,510,0,50,12,"2020-08-16"),
        (10,382,5,34,0,"2020-08-17"),(10,628,0,0,21,"2020-08-18"),(10,428,4,22,37,"2020-08-19"),(10,413,1,12,34,"2020-08-20"),
        (10,318,6,6,7,"2020-08-21"),(10,700,2,16,30,"2020-08-22"),(10,1021,5,11,32,"2020-08-23"),(10,284,8,0,28,"2020-08-24"),
        (10,492,2,3,6,"2020-08-25"),(10,238,0,21,16,"2020-08-26"),(10,453,0,0,17,"2020-08-27"),(10,333,10,6,14,"2020-08-28"),
        (10,609,0,14,16,"2020-08-29"),(10,395,5,0,7,"2020-08-30"),(10,530,1,9,0,"2020-08-31"),
        (11,2134,0,8,11,"2020-08-01"),(11,1093,0,18,12,"2020-08-02"),(11,1467,0,15,2,"2020-08-03"),(11,1760,0,4,9,"2020-08-04"),
        (11,1482,0,12,0,"2020-08-05"),(11,1004,0,0,2,"2020-08-06"),(11,2402,1,0,8,"2020-08-07"),(11,838,1,0,0,"2020-08-08"),
        (11,922,0,0,0,"2020-08-09"),(11,983,2,2,3,"2020-08-10"),(11,1803,1,7,10,"2020-08-11"),(11,927,0,12,4,"2020-08-12"),
        (11,1450,1,16,3,"2020-08-13"),(11,889,0,7,9,"2020-08-14"),(11,1234,0,28,0,"2020-08-15"),(11,1583,0,17,0,"2020-08-16"),
        (11,1503,0,0,0,"2020-08-17"),(11,798,1,0,3,"2020-08-18"),(11,1652,0,0,0,"2020-08-19"),(11,1362,0,0,6,"2020-08-20"),
        (11,1793,0,0,0,"2020-08-21"),(11,1928,0,0,0,"2020-08-22"),(11,821,0,7,11,"2020-08-23"),(11,1278,1,0,0,"2020-08-24"),
        (11,1112,0,7,18,"2020-08-25"),(11,1356,0,19,4,"2020-08-26"),(11,629,2,15,0,"2020-08-27"),(11,1029,2,8,13,"2020-08-28"),
        (11,829,0,4,8,"2020-08-29"),(11,1224,3,0,0,"2020-08-30"),(11,492,0,6,8,"2020-08-31"),
        (12,92,5,32,30,"2020-08-01"),(12,201,1,21,17,"2020-08-02"),(12,218,12,8,13,"2020-08-03"),(12,49,2,21,4,"2020-08-04"),
        (12,134,8,16,40,"2020-08-05"),(12,192,8,19,23,"2020-08-06"),(12,167,15,9,6,"2020-08-07"),(12,92,12,24,16,"2020-08-08"),
        (12,27,1,39,21,"2020-08-09"),(12,78,17,27,15,"2020-08-10"),(12,51,8,17,8,"2020-08-11"),(12,106,14,12,13,"2020-08-12"),
        (12,39,5,27,72,"2020-08-13"),(12,90,4,18,31,"2020-08-14"),(12,73,9,10,0,"2020-08-15"),(12,176,12,9,7,"2020-08-16"),
        (12,182,0,17,52,"2020-08-17"),(12,91,8,12,8,"2020-08-18"),(12,82,7,0,6,"2020-08-19"),(12,234,6,0,15,"2020-08-20"),
        (12,76,9,10,29,"2020-08-21"),(12,108,6,18,8,"2020-08-22"),(12,72,9,14,25,"2020-08-23"),(12,188,2,7,8,"2020-08-24"),
        (12,111,13,11,17,"2020-08-25"),(12,115,0,7,9,"2020-08-26"),(12,63,6,30,4,"2020-08-27"),(12,177,0,17,22,"2020-08-28"),
        (12,124,12,20,28,"2020-08-29"),(12,100,3,0,13,"2020-08-30"),(12,94,14,19,13,"2020-08-31"),  
        (13,530,5,15,12,"2020-08-01"),(13,736,4,0,0,"2020-08-02"),(13,649,4,13,22,"2020-08-03"),(13,698,6,0,0,"2020-08-04"),
        (13,721,2,17,23,"2020-08-05"),(13,816,2,18,6,"2020-08-06"),(13,701,3,19,18,"2020-08-07"),(13,726,7,0,17,"2020-08-08"),
        (13,394,3,9,18,"2020-08-09"),(13,793,0,9,2,"2020-08-10"),(13,612,9,0,5,"2020-08-11"),(13,746,2,14,8,"2020-08-12"),
        (13,455,1,0,14,"2020-08-13"),(13,439,1,21,7,"2020-08-14"),(13,682,8,7,9,"2020-08-15"),(13,893,0,19,23,"2020-08-16"),
        (13,801,0,13,0,"2020-08-17"),(13,399,7,2,15,"2020-08-18"),(13,723,0,16,15,"2020-08-19"),(13,487,0,21,22,"2020-08-20"),
        (13,739,6,20,8,"2020-08-21"),(13,465,6,3,5,"2020-08-22"),(13,788,3,7,13,"2020-08-23"),(13,589,5,12,6,"2020-08-24"),
        (13,666,2,13,3,"2020-08-25"),(13,582,0,0,16,"2020-08-26"),(13,625,7,12,12,"2020-08-27"),(13,501,3,4,8,"2020-08-28"),
        (13,615,0,9,14,"2020-08-29"),(13,611,3,4,16,"2020-08-30"),(13,791,4,9,28,"2020-08-31"),
        (14,983,1,34,61,"2020-08-01"),(14,999,1,78,32,"2020-08-02"),(14,1039,0,34,19,"2020-08-03"),(14,772,1,53,31,"2020-08-04"),
        (14,1023,0,29,27,"2020-08-05"),(14,983,0,60,31,"2020-08-06"),(14,1324,0,28,29,"2020-08-07"),(14,861,3,49,30,"2020-08-08"),
        (14,1034,0,38,42,"2020-08-09"),(14,836,0,0,46,"2020-08-10"),(14,927,0,35,44,"2020-08-11"),(14,697,2,26,8,"2020-08-12"),
        (14,802,0,55,12,"2020-08-13"),(14,782,0,13,34,"2020-08-14"),(14,915,1,50,18,"2020-08-15"),(14,810,0,20,17,"2020-08-16"),
        (14,709,2,71,22,"2020-08-17"),(14,888,2,12,19,"2020-08-18"),(14,827,0,38,33,"2020-08-19"),(14,1293,0,9,29,"2020-08-20"),
        (14,1823,0,59,0,"2020-08-21"),(14,1083,0,23,12,"2020-08-22"),(14,1381,1,29,19,"2020-08-23"),(14,1173,1,13,39,"2020-08-24"),
        (14,1432,0,24,47,"2020-08-25"),(14,1063,0,42,19,"2020-08-26"),(14,1402,0,33,59,"2020-08-27"),(14,929,0,13,74,"2020-08-28"),
        (14,1304,0,14,59,"2020-08-29"),(14,1145,3,33,30,"2020-08-30"),(14,1102,0,72,0,"2020-08-31"),
        (15,539,3,13,26,"2020-08-01"),(15,617,0,27,14,"2020-08-02"),(15,729,0,18,10,"2020-08-03"),(15,521,1,0,24,"2020-08-04"),
        (15,491,0,14,22,"2020-08-05"),(15,719,4,7,27,"2020-08-06"),(15,821,0,8,11,"2020-08-07"),(15,594,1,32,7,"2020-08-08"),
        (15,928,4,8,16,"2020-08-09"),(15,830,3,0,16,"2020-08-10"),(15,902,3,0,6,"2020-08-11"),(15,492,4,22,12,"2020-08-12"),
        (15,619,6,9,14,"2020-08-13"),(15,715,2,25,2,"2020-08-14"),(15,762,5,12,1,"2020-08-15"),(15,378,7,3,24,"2020-08-16"),
        (15,729,8,17,0,"2020-08-17"),(15,785,6,4,8,"2020-08-18"),(15,681,12,17,27,"2020-08-19"),(15,402,5,8,2,"2020-08-20"),
        (15,800,2,0,6,"2020-08-21"),(15,516,8,0,0,"2020-08-22"),(15,535,3,9,5,"2020-08-23"),(15,501,4,7,1,"2020-08-24"),
        (15,910,1,2,6,"2020-08-25"),(15,684,9,21,9,"2020-08-26"),(15,587,2,18,12,"2020-08-27"),(15,610,2,14,18,"2020-08-28"),
        (15,527,5,16,0,"2020-08-29"),(15,901,2,13,15,"2020-08-30"),(15,534,0,6,14,"2020-08-31"),
        (17,1648,0,3,0,"2020-08-01"),(17,1849,0,6,11,"2020-08-02"),(17,1453,0,12,10,"2020-08-03"),(17,921,0,13,5,"2020-08-04"),
        (17,1623,0,0,0,"2020-08-05"),(17,1632,0,0,0,"2020-08-06"),(17,1829,0,2,9,"2020-08-07"),(17,1834,1,0,5,"2020-08-08"),
        (17,2341,0,0,7,"2020-08-09"),(17,2483,2,0,8,"2020-08-10"),(17,1712,1,2,3,"2020-08-11"),(17,1749,0,12,9,"2020-08-12"),
        (17,1232,0,0,0,"2020-08-13"),(17,2013,0,2,0,"2020-08-14"),(17,1144,1,4,7,"2020-08-15"),(17,1334,2,21,0,"2020-08-16"),
        (17,1552,0,0,0,"2020-08-17"),(17,1934,0,3,5,"2020-08-18"),(17,1536,0,11,2,"2020-08-19"),(17,1346,0,3,1,"2020-08-20"),
        (17,920,1,8,6,"2020-08-21"),(17,1235,1,10,2,"2020-08-22"),(17,2184,1,13,8,"2020-08-23"),(17,1765,3,6,8,"2020-08-24"),
        (17,1928,0,5,4,"2020-08-25"),(17,1321,0,5,1,"2020-08-26"),(17,2562,0,0,18,"2020-08-27"),(17,1113,0,11,11,"2020-08-28"),
        (17,1057,0,3,0,"2020-08-29"),(17,1022,0,0,7,"2020-08-30"),(17,1240,0,0,0,"2020-08-31"),
        (18,782,0,46,54,"2020-08-01"),(18,492,6,40,31,"2020-08-02"),(18,362,1,49,28,"2020-08-03"),(18,593,7,12,34,"2020-08-04"),
        (18,892,2,24,62,"2020-08-05"),(18,584,3,4,52,"2020-08-06"),(18,392,0,21,35,"2020-08-07"),(18,793,1,29,29,"2020-08-08"),
        (18,1639,4,32,23,"2020-08-09"),(18,493,0,62,14,"2020-08-10"),(18,418,4,44,49,"2020-08-11"),(18,632,0,85,24,"2020-08-12"),
        (18,622,1,18,27,"2020-08-13"),(18,302,4,21,19,"2020-08-14"),(18,472,2,20,38,"2020-08-15"),(18,535,5,37,19,"2020-08-16"),
        (18,522,5,9,49,"2020-08-17"),(18,592,7,42,35,"2020-08-18"),(18,520,3,19,37,"2020-08-19"),(18,589,5,20,29,"2020-08-20"),
        (18,482,6,19,18,"2020-08-21"),(18,857,4,19,72,"2020-08-22"),(18,539,8,4,13,"2020-08-23"),(18,470,1,44,31,"2020-08-24"),
        (18,527,2,49,38,"2020-08-25"),(18,496,0,23,32,"2020-08-26"),(18,611,6,39,19,"2020-08-27"),(18,593,2,54,40,"2020-08-28"),
        (18,812,0,75,21,"2020-08-29"),(18,523,2,29,42,"2020-08-30"),(18,389,0,82,14,"2020-08-31"),
        (19,1345,1,18,21,"2020-08-01"),(19,1520,3,20,18,"2020-08-02"),(19,1037,4,19,12,"2020-08-03"),(19,1628,0,18,11,"2020-08-04"),
        (19,1129,0,14,7,"2020-08-05"),(19,1045,0,19,19,"2020-08-06"),(19,1122,3,14,15,"2020-08-07"),(19,1072,1,23,14,"2020-08-08"),
        (19,1249,2,19,17,"2020-08-09"),(19,1294,4,14,16,"2020-08-10"),(19,972,0,20,13,"2020-08-11"),(19,882,0,19,8,"2020-08-12"),
        (19,938,2,20,8,"2020-08-13"),(19,1183,0,9,8,"2020-08-14"),(19,927,2,28,19,"2020-08-15"),(19,926,1,10,7,"2020-08-16"),
        (19,910,0,11,16,"2020-08-17"),(19,1009,3,15,20,"2020-08-18"),(19,899,0,7,9,"2020-08-19"),(19,935,4,12,19,"2020-08-20"),
        (19,888,0,12,22,"2020-08-21"),(19,1212,2,11,18,"2020-08-22"),(19,1322,3,11,15,"2020-08-23"),(19,1088,2,6,13,"2020-08-24"),
        (19,1420,4,21,29,"2020-08-25"),(19,988,0,17,9,"2020-08-26"),(19,1129,0,8,10,"2020-08-27"),(19,1333,0,17,16,"2020-08-28"),
        (19,1333,1,27,19,"2020-08-29"),(19,892,1,10,4,"2020-08-30"),(19,1341,2,13,9,"2020-08-31"),
        (21,1483,0,53,34,"2020-08-01"),(21,1938,0,34,42,"2020-08-02"),(21,1305,0,78,22,"2020-08-03"),(21,1324,0,25,39,"2020-08-04"),
        (21,1837,0,32,28,"2020-08-05"),(21,2183,0,21,38,"2020-08-06"),(21,1735,1,26,43,"2020-08-07"),(21,1253,3,41,19,"2020-08-08"),
        (21,983,1,35,20,"2020-08-09"),(21,1634,3,17,29,"2020-08-10"),(21,1538,0,33,19,"2020-08-11"),(21,1653,0,55,67,"2020-08-12"),
        (21,1038,0,21,17,"2020-08-13"),(21,826,0,20,17,"2020-08-14"),(21,1492,1,82,34,"2020-08-15"),(21,1403,0,31,56,"2020-08-16"),
        (21,1392,0,27,19,"2020-08-17"),(21,1542,0,17,34,"2020-08-18"),(21,937,0,77,40,"2020-08-19"),(21,1839,0,20,42,"2020-08-20"),
        (21,1837,0,31,30,"2020-08-21"),(21,1639,0,32,46,"2020-08-22"),(21,827,1,23,37,"2020-08-23"),(21,1432,1,45,39,"2020-08-24"),
        (21,803,0,14,52,"2020-08-25"),(21,1329,0,82,52,"2020-08-26"),(21,1748,0,81,12,"2020-08-27"),(21,1522,1,50,50,"2020-08-28"),
        (21,1432,2,39,42,"2020-08-29"),(21,1238,1,68,51,"2020-08-30"),(21,2242,0,54,32,"2020-08-31"),
        (22,2403,0,52,43,"2020-08-01"),(22,2912,0,61,30,"2020-08-02"),(22,1098,1,83,21,"2020-08-03"),(22,2183,2,61,39,"2020-08-04"),
        (22,2421,0,19,37,"2020-08-05"),(22,2182,1,18,29,"2020-08-06"),(22,1128,0,66,55,"2020-08-07"),(22,2382,0,53,48,"2020-08-08"),
        (22,2018,0,22,39,"2020-08-09"),(22,1592,0,28,27,"2020-08-10"),(22,829,0,47,30,"2020-08-11"),(22,2110,0,20,20,"2020-08-12"),
        (22,1732,0,49,28,"2020-08-13"),(22,1037,0,68,21,"2020-08-14"),(22,1638,0,29,17,"2020-08-15"),(22,1044,0,16,44,"2020-08-16"),
        (22,1731,1,8,22,"2020-08-17"),(22,1111,0,72,19,"2020-08-18"),(22,1543,0,35,39,"2020-08-19"),(22,937,0,59,21,"2020-08-20"),
        (22,1372,0,3,53,"2020-08-21"),(22,1762,0,18,35,"2020-08-22"),(22,1472,0,13,28,"2020-08-23"),(22,2493,0,20,46,"2020-08-24"),
        (22,1138,0,13,42,"2020-08-25"),(22,1662,0,9,13,"2020-08-26"),(22,1600,0,95,8,"2020-08-27"),(22,891,0,39,53,"2020-08-28"),
        (22,1240,0,72,31,"2020-08-29"),(22,1144,0,23,38,"2020-08-30"),(22,1593,1,52,29,"2020-08-31"),
        (23,222,2,50,55,"2020-08-01"),(23,62,0,60,32,"2020-08-02"),(23,26,0,30,22,"2020-08-03"),(23,29,1,7,67,"2020-08-04"),
        (23,169,1,62,42,"2020-08-05"),(23,173,0,47,51,"2020-08-06"),(23,135,0,18,48,"2020-08-07"),(23,99,0,50,15,"2020-08-08"),
        (23,82,0,38,46,"2020-08-09"),(23,200,0,23,30,"2020-08-10"),(23,147,0,21,49,"2020-08-11"),(23,178,0,42,21,"2020-08-12"),
        (23,183,0,30,51,"2020-08-13"),(23,144,0,28,36,"2020-08-14"),(23,168,0,30,30,"2020-08-15"),(23,299,2,30,8,"2020-08-16"),
        (23,283,0,41,29,"2020-08-17"),(23,152,2,19,45,"2020-08-18"),(23,291,0,34,31,"2020-08-19"),(23,219,1,18,19,"2020-08-20"),
        (23,212,0,39,44,"2020-08-21"),(23,169,0,29,39,"2020-08-22"),(23,53,0,29,34,"2020-08-23"),(23,178,0,40,49,"2020-08-24"),
        (23,192,1,50,28,"2020-08-25"),(23,98,0,25,45,"2020-08-26"),(23,22,1,32,85,"2020-08-27"),(23,155,0,33,51,"2020-08-28"),
        (23,73,0,40,46,"2020-08-29"),(23,54,1,14,25,"2020-08-30"),(23,35,3,12,90,"2020-08-31"),
        (25,1839,4,42,19,"2020-08-01"),(25,1555,0,60,6,"2020-08-02"),(25,1412,0,21,7,"2020-08-03"),(25,1242,5,29,14,"2020-08-04"),
        (25,1537,2,32,17,"2020-08-05"),(25,1492,1,32,0,"2020-08-06"),(25,1500,1,27,29,"2020-08-07"),(25,1039,2,37,28,"2020-08-08"),
        (25,1392,0,59,8,"2020-08-09"),(25,1637,0,43,12,"2020-08-10"),(25,902,1,22,49,"2020-08-11"),(25,1183,0,81,8,"2020-08-12"),
        (25,892,3,24,13,"2020-08-13"),(25,1228,3,50,32,"2020-08-14"),(25,1182,4,38,7,"2020-08-15"),(25,1340,0,70,21,"2020-08-16"),
        (25,989,0,9,17,"2020-08-17"),(25,1447,0,72,12,"2020-08-18"),(25,1632,0,83,11,"2020-08-19"),(25,1483,3,63,18,"2020-08-20"),
        (25,1892,4,47,20,"2020-08-21"),(25,963,0,52,27,"2020-08-22"),(25,1701,0,28,17,"2020-08-23"),(25,1204,4,57,19,"2020-08-24"),
        (25,792,1,56,19,"2020-08-25"),(25,1284,0,43,12,"2020-08-26"),(25,1132,5,32,19,"2020-08-27"),(25,1874,1,48,2,"2020-08-28"),
        (25,1638,0,72,11,"2020-08-29"),(25,1195,4,50,22,"2020-08-30"),(25,1233,2,58,4,"2020-08-31"),
        (26,412,5,59,42,"2020-08-01"),(26,298,5,77,45,"2020-08-02"),(26,429,7,34,47,"2020-08-03"),(26,404,8,44,60,"2020-08-04"),
        (26,383,13,38,30,"2020-08-05"),(26,288,7,61,15,"2020-08-06"),(26,502,4,17,50,"2020-08-07"),(26,593,6,29,36,"2020-08-08"),
        (26,492,9,28,24,"2020-08-09"),(26,232,13,21,41,"2020-08-10"),(26,489,3,20,29,"2020-08-11"),(26,605,4,39,52,"2020-08-12"),
        (26,502,2,35,41,"2020-08-13"),(26,399,15,23,33,"2020-08-14"),(26,400,3,40,33,"2020-08-15"),(26,472,13,35,49,"2020-08-16"),
        (26,664,8,18,33,"2020-08-17"),(26,477,6,41,38,"2020-08-18"),(26,302,7,67,28,"2020-08-19"),(26,533,2,58,30,"2020-08-20"),
        (26,544,11,55,25,"2020-08-21"),(26,425,3,57,29,"2020-08-22"),(26,239,14,50,33,"2020-08-23"),(26,421,6,62,43,"2020-08-24"),
        (26,372,13,39,59,"2020-08-25"),(26,362,1,14,40,"2020-08-26"),(26,277,3,60,48,"2020-08-27"),(26,388,10,50,50,"2020-08-28"),
        (26,301,8,78,30,"2020-08-29"),(26,311,9,92,3,"2020-08-30"),(26,399,2,67,21,"2020-08-31"),
        (27,183,5,15,37,"2020-08-01"),(27,283,6,3,29,"2020-08-02"),(27,156,9,33,25,"2020-08-03"),(27,253,14,8,39,"2020-08-04"),
        (27,200,7,22,28,"2020-08-05"),(27,109,8,16,21,"2020-08-06"),(27,161,7,18,48,"2020-08-07"),(27,200,5,12,62,"2020-08-08"),
        (27,126,4,13,31,"2020-08-09"),(27,193,0,7,36,"2020-08-10"),(27,98,5,16,53,"2020-08-11"),(27,121,3,17,30,"2020-08-12"),
        (27,255,9,28,20,"2020-08-13"),(27,244,9,37,24,"2020-08-14"),(27,188,11,21,37,"2020-08-15"),(27,249,6,24,24,"2020-08-16"),
        (27,242,1,31,23,"2020-08-17"),(27,237,15,15,57,"2020-08-18"),(27,264,6,7,54,"2020-08-19"),(27,255,1,41,19,"2020-08-20"),
        (27,251,13,7,44,"2020-08-21"),(27,281,2,30,34,"2020-08-22"),(27,338,3,23,43,"2020-08-23"),(27,201,13,8,57,"2020-08-24"),
        (27,277,11,13,38,"2020-08-25"),(27,299,12,28,19,"2020-08-26"),(27,139,1,22,47,"2020-08-27"),(27,267,10,16,40,"2020-08-28"),
        (27,178,8,19,27,"2020-08-29"),(27,177,17,9,45,"2020-08-30"),(27,87,12,11,18,"2020-08-31"),
        (28,1124,14,5,12,"2020-08-01"),(28,1044,7,7,7,"2020-08-02"),(28,755,13,6,6,"2020-08-03"),(28,1533,18,7,2,"2020-08-04"),
        (28,679,18,8,4,"2020-08-05"),(28,677,11,1,8,"2020-08-06"),(28,632,16,7,0,"2020-08-07"),(28,1034,7,3,3,"2020-08-08"),
        (28,575,2,14,4,"2020-08-09"),(28,732,15,25,0,"2020-08-10"),(28,687,2,3,15,"2020-08-11"),(28,543,9,11,16,"2020-08-12"),
        (28,533,7,4,9,"2020-08-13"),(28,789,4,19,17,"2020-08-14"),(28,555,1,14,9,"2020-08-15"),(28,566,12,13,27,"2020-08-16"),
        (28,423,14,3,5,"2020-08-17"),(28,643,9,7,2,"2020-08-18"),(28,502,7,5,5,"2020-08-19"),(28,624,13,1,17,"2020-08-20"),
        (28,1463,5,0,11,"2020-08-21"),(28,611,12,8,11,"2020-08-22"),(28,487,14,3,16,"2020-08-23"),(28,620,8,22,0,"2020-08-24"),
        (28,699,11,14,8,"2020-08-25"),(28,586,10,17,0,"2020-08-26"),(28,679,9,0,4,"2020-08-27"),(28,1111,5,9,6,"2020-08-28"),
        (28,581,8,21,10,"2020-08-29"),(28,581,6,4,13,"2020-08-30"),(28,700,6,7,13,"2020-08-31"),
        (29,1524,0,56,25,"2020-08-01"),(29,987,0,42,39,"2020-08-02"),(29,1439,1,34,38,"2020-08-03"),(29,1188,0,41,37,"2020-08-04"),
        (29,1730,0,48,39,"2020-08-05"),(29,1771,0,49,43,"2020-08-06"),(29,1378,0,41,29,"2020-08-07"),(29,1263,0,33,59,"2020-08-08"),
        (29,1930,1,27,57,"2020-08-09"),(29,1849,1,38,36,"2020-08-10"),(29,1598,0,39,37,"2020-08-11"),(29,1730,0,21,68,"2020-08-12"),
        (29,1847,0,64,19,"2020-08-13"),(29,1667,0,30,68,"2020-08-14"),(29,1702,0,68,31,"2020-08-15"),(29,2740,0,28,46,"2020-08-16"),
        (29,1222,0,30,34,"2020-08-17"),(29,1493,4,27,59,"2020-08-18"),(29,1832,2,84,18,"2020-08-19"),(29,2489,0,34,32,"2020-08-20"),
        (29,1352,2,74,40,"2020-08-21"),(29,1339,0,18,22,"2020-08-22"),(29,1440,0,36,36,"2020-08-23"),(29,2009,2,50,56,"2020-08-24"),
        (29,1667,0,55,36,"2020-08-25"),(29,1028,0,37,65,"2020-08-26"),(29,927,0,25,39,"2020-08-27"),(29,2021,0,33,13,"2020-08-28"),
        (29,1599,0,61,38,"2020-08-29"),(29,1088,0,70,39,"2020-08-30"),(29,1073,1,47,27,"2020-08-31"),
        (30,329,2,20,45,"2020-08-01"),(30,222,8,10,47,"2020-08-02"),(30,173,12,4,41,"2020-08-03"),(30,177,13,7,52,"2020-08-04"),
        (30,710,6,10,42,"2020-08-05"),(30,334,2,2,37,"2020-08-06"),(30,123,16,8,39,"2020-08-07"),(30,189,15,8,46,"2020-08-08"),
        (30,400,5,7,29,"2020-08-09"),(30,354,6,20,78,"2020-08-10"),(30,394,9,6,79,"2020-08-11"),(30,207,9,11,83,"2020-08-12"),
        (30,329,0,4,47,"2020-08-13"),(30,378,6,4,44,"2020-08-14"),(30,338,14,7,39,"2020-08-15"),(30,284,2,6,93,"2020-08-16"),
        (30,284,0,6,36,"2020-08-17"),(30,312,13,1,39,"2020-08-18"),(30,267,17,6,29,"2020-08-19"),(30,351,8,15,43,"2020-08-20"),
        (30,244,4,0,67,"2020-08-21"),(30,200,4,14,29,"2020-08-22"),(30,313,11,12,58,"2020-08-23"),(30,273,14,14,55,"2020-08-24"),
        (30,285,12,11,39,"2020-08-25"),(30,173,5,9,55,"2020-08-26"),(30,461,10,13,64,"2020-08-27"),(30,290,17,29,61,"2020-08-28"),
        (30,197,7,8,51,"2020-08-29"),(30,183,15,4,64,"2020-08-30"),(30,245,16,50,43,"2020-08-31")]

df_consumos_diarios = spark.createDataFrame(data, columns)
df_consumos_diarios.write.mode("overwrite").saveAsTable('df_consumos_diarios')

**1. Mostrar por pantalla el número TOTAL de clientes que tenían en el mes anterior más de un contrato con la compañía**

In [0]:
# Vamos a necesitar la tabla de facturas del mes anterior. Para identificar y distinguir cada cliente, es suficiente con atender al id_cliente, pero para averiguar el número de contratos que pertenece a cada cliente, deberemos contabilizar todos aquellos registros que contengan el mismo id_cliente. Para ello utilizaremos una operación "Window" en función del campo id_cliente, que, de forma similar a la función "groupBy", nos agrupará las filas que compartan el mismo id_cliente, pero con la particularidad de que no eliminará ningún registro.

# Cuando definimos una ventana por la que hacer una partición, tiene que estar obligatoriamente ordenada por alguna de las columnas. En este caso ordenaremos por la columna "importe" en orden descendente, para que nos muestre el primero el contrato más caro que el cliente tiene con nuestra compañía.


df_facturas_mes_ant = spark.read.table("df_facturas_mes_ant")

window_cliente = Window.partitionBy("id_cliente").orderBy(desc("importe"))

# utilizamos el comando withColumn para crear una nueva columna llamada "count_contratos", que contendrá el resultado del comando row_number(), que contará el número de registros de cada partición Window.
df_facturas_mes_ant = df_facturas_mes_ant.withColumn("count_contratos", row_number().over(window_cliente))\
                                         .withColumn("total_contratos", max("count_contratos").over(window_cliente.rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing))).orderBy("id_cliente")

# Como puede haber ventanas con más de 2 elementos (por ejemplo el caso del id_cliente 1), creamos otra columna ("total_contratos") que nos informe del número maximo de la columna que acabamos de crear ("count_contratos").
                                        
df_facturas_mes_ant.show()

+----------+---------+-------+----------+---------------+---------------+
id_cliente|id_oferta|importe| fecha|count_contratos|total_contratos|
+----------+---------+-------+----------+---------------+---------------+
 1| 6| 124.5|2020-07-31| 1| 3|
 1| 6| 118.6|2020-07-31| 2| 3|
 1| 9| 36.5|2020-07-31| 3| 3|
 2| 4| 95.0|2020-07-31| 1| 1|
 3| 5| 102.5|2020-07-31| 1| 1|
 5| 7| 80.5|2020-07-31| 1| 2|
 5| 9| 40.5|2020-07-31| 2| 2|
 6| 8| 84.0|2020-07-31| 1| 1|
 7| 4| 95.0|2020-07-31| 1| 1|
 8| 5| 102.5|2020-07-31| 1| 1|
 10| 6| 124.5|2020-07-31| 1| 1|
 11| 3| 80.0|2020-07-31| 1| 1|
 12| 1| 70.0|2020-07-31| 1| 1|
 13| 4| 95.0|2020-07-31| 1| 1|
 14| 9| 42.5|2020-07-31| 1| 1|
 15| 6| 115.0|2020-07-31| 1| 2|
 15| 2| 78.5|2020-07-31| 2| 2|
 16| 1| 70.0|2020-07-31| 1| 1|
 17| 8| 84.0|2020-07-31| 1| 1|
 18| 5| 102.5|2020-07-31| 1| 2|
+----------+---------+-------+----------+---------------+---------------+
only showing top 20 rows

In [0]:
# Ahora podemos filtrar aquellas filas en las que coincidan count_contratos y total_contratos, y que además count_contratos sea mayor que 1 para obtener aquellos clientes del mes anterior que tenían más de un contrato. 
# A esta misma transformación le aplicamos el comando count() y obtenemos el número total de clientes con más de un contrato en el mes anterior.

df_clientes_vip = df_facturas_mes_ant.filter((col("count_contratos") == col("total_contratos")) & (col("count_contratos") > 1))
df_clientes_vip.show()

print("El número TOTAL de clientes que tenían en el mes anterior más de un contrato con la compañía es: " + str(df_clientes_vip.count()))

+----------+---------+-------+----------+---------------+---------------+
id_cliente|id_oferta|importe| fecha|count_contratos|total_contratos|
+----------+---------+-------+----------+---------------+---------------+
 1| 9| 36.5|2020-07-31| 3| 3|
 5| 9| 40.5|2020-07-31| 2| 2|
 15| 2| 78.5|2020-07-31| 2| 2|
 18| 9| 37.5|2020-07-31| 2| 2|
 22| 7| 69.5|2020-07-31| 2| 2|
 25| 10| 31.99|2020-07-31| 2| 2|
 27| 9| 41.5|2020-07-31| 2| 2|
 30| 2| 70.0|2020-07-31| 2| 2|
+----------+---------+-------+----------+---------------+---------------+

El número TOTAL de clientes que tenían en el mes anterior más de un contrato con la compañía es: 8

**2. Generar un nuevo dataframe de facturas del mes actual que asigne un 7% de descuento a todos los contratos de clientes que ya existían en el mes anterior (los contratos de los clientes nuevos seguirán con el mismo importe). Mantener la columna "importe" y crear una nueva columna "importe_dto" con el nuevo importe, casteada a 2 decimales. Mostrar además el resultado ordenado por los campos id_cliente ascendente, importe descendente. **

In [0]:
# En este caso vamos a necesitar las tablas de facturas del mes actual y del mes anterior para obtener los contratos de los clientes que han continuado en el tiempo, que son a los que vamos a aplicarles el descuento.

df_facturas_mes_ant = spark.read.table("df_facturas_mes_ant")
df_facturas_mes_actual = spark.read.table("df_facturas_mes_actual")


Para identificar los contratos del mes actual que ya existían en el mes anterior, con los campos facilitados, tenemos dos posibles interpretaciones:

   1.  Estamos ante el mismo contrato cuando el _id_cliente_ , _id_oferta_ e _importe_ coinciden: Es aplicable a la mayoría de los registros, ejemplo, los dos registros del id_cliente 1 con id_oferta 6 (124.5 y 118.6 €), aunque existen algunos registros que no, como el id_cliente 8, oferta 5, que existe en ambas tablas pero con diferente importe.
        
   2.  Estamos ante el mismo contrato cuando el *id_cliente* , *id_oferta* coinciden aunque el importe pueda cambiar de una tabla a otra: Esta interpretación no deja casos fuera pero, nos dificulta identificar que contratos son exactamente los mismos, y por  tanto a cuales aplicaremos el descuento.


En el primer caso, nos bastaria con ejecutar un inner join sobre  "id_cliente", "id_oferta", "importe", en el segundo caso, necesitaremos crear una columna adicional en ambas tablas que nos ayude a identificar cada contrato junto con las claves "id_cliente", "id_oferta".

Para ello, para crear esta columna (que llamaremos "count_contratos"), implementaremos una partición sobre id_cliente e id_oferta y aplicaremos el comando **rownumber()**, que contará todos los registros que contengan el mismo id_cliente y mismo id_oferta.

In [0]:
winSpec_cliente_oferta = Window.partitionBy("id_cliente","id_oferta").orderBy(asc("id_cliente"), desc("importe"))

df_facturas_mes_actual_count = df_facturas_mes_actual.withColumn("count_contratos", row_number().over(winSpec_cliente_oferta)).orderBy(asc("id_cliente"), desc("importe"))
df_facturas_mes_ant_count = df_facturas_mes_ant.withColumn("count_contratos", row_number().over(winSpec_cliente_oferta)).orderBy(asc("id_cliente"), desc("importe"))


df_facturas_mes_actual_count.show(5)
df_facturas_mes_ant_count.show(5)

+----------+---------+-------+----------+---------------+
id_cliente|id_oferta|importe| fecha|count_contratos|
+----------+---------+-------+----------+---------------+
 1| 6| 124.5|2020-08-31| 1|
 1| 6| 118.6|2020-08-31| 2|
 2| 4| 95.0|2020-08-31| 1|
 3| 5| 102.5|2020-08-31| 1|
 4| 1| 70.0|2020-08-31| 1|
+----------+---------+-------+----------+---------------+
only showing top 5 rows

+----------+---------+-------+----------+---------------+
id_cliente|id_oferta|importe| fecha|count_contratos|
+----------+---------+-------+----------+---------------+
 1| 6| 124.5|2020-07-31| 1|
 1| 6| 118.6|2020-07-31| 2|
 1| 9| 36.5|2020-07-31| 1|
 2| 4| 95.0|2020-07-31| 1|
 3| 5| 102.5|2020-07-31| 1|
+----------+---------+-------+----------+---------------+
only showing top 5 rows

In [0]:
# En este caso, en vez de agregar información de la tabla de facturas del mes anterior a la del mes actual, lo que queremos es aplicar un tratamiento solo a una parte de los registros, a aquellos contratos que ya existían en el mes anterior y que siguen existiendo en este mes, pero al mismo tiempo queremos mantener el resto de registros de la tabla de facturas del mes actual.

# Para ello es necesario hacer un left join también, pero incluimos una columna auxiliar en el DF del mes anterior para saber que registros han hecho match entre las dos tablas (los que no hayan hecho match tendrán un valor "null" en la columna auxiliar). Con esta columna, luego será posible aplicar el tratamiento sólo a los contratos deseados.

df_facturas_mes_actual_flg = df_facturas_mes_actual_count.join(df_facturas_mes_ant_count.select("id_cliente", "id_oferta", "count_contratos")\
                                                         .withColumn("flg_mes_ant", lit(1)),["id_cliente", "id_oferta", "count_contratos"],"left") 
                                                                            

df_facturas_mes_actual_flg.show()                                 

+----------+---------+---------------+-------+----------+-----------+
id_cliente|id_oferta|count_contratos|importe| fecha|flg_mes_ant|
+----------+---------+---------------+-------+----------+-----------+
 1| 6| 1| 124.5|2020-08-31| 1|
 1| 6| 2| 118.6|2020-08-31| 1|
 2| 4| 1| 95.0|2020-08-31| 1|
 3| 5| 1| 102.5|2020-08-31| 1|
 4| 1| 1| 70.0|2020-08-31| null|
 5| 7| 1| 80.5|2020-08-31| 1|
 5| 9| 1| 40.5|2020-08-31| 1|
 6| 8| 1| 84.0|2020-08-31| 1|
 7| 4| 1| 95.0|2020-08-31| 1|
 8| 5| 1| 100.5|2020-08-31| 1|
 9| 7| 1| 78.5|2020-08-31| null|
 9| 10| 1| 29.99|2020-08-31| null|
 10| 6| 1| 124.5|2020-08-31| 1|
 11| 3| 1| 83.0|2020-08-31| 1|
 12| 1| 1| 70.0|2020-08-31| 1|
 13| 4| 1| 95.0|2020-08-31| 1|
 14| 9| 1| 42.5|2020-08-31| 1|
 15| 6| 1| 115.0|2020-08-31| 1|
 15| 2| 1| 78.5|2020-08-31| 1|
 17| 8| 1| 84.0|2020-08-31| 1|
+----------+---------+---------------+-------+----------+-----------+
only showing top 20 rows

In [0]:
# Ahora procederemos a aplicar el descuento a aquellos contratos que tienen 1 en el campo "flg_mes_ant" (los que han hechos match). Crearemos la columna "importe_dto" que contendrá el nuevo importe con descuento aplicado, casteado a 2 decimales y ordenando el df por id_cliente ascendente, importe descendente.


df_facturas_mes_actual_descuento = df_facturas_mes_actual_flg.withColumn("importe_dto", when(col("flg_mes_ant") == 1, col("importe") * 0.93)\
                                                                         .otherwise(col("importe")))\
                                                                         .withColumn("importe_dto", col("importe_dto").cast(DecimalType(10,2)))\
                                                                         .orderBy(asc("id_cliente"), desc("importe"))

df_facturas_mes_actual_descuento.show()

+----------+---------+---------------+-------+----------+-----------+-----------+
id_cliente|id_oferta|count_contratos|importe| fecha|flg_mes_ant|importe_dto|
+----------+---------+---------------+-------+----------+-----------+-----------+
 1| 6| 1| 124.5|2020-08-31| 1| 115.79|
 1| 6| 2| 118.6|2020-08-31| 1| 110.30|
 2| 4| 1| 95.0|2020-08-31| 1| 88.35|
 3| 5| 1| 102.5|2020-08-31| 1| 95.33|
 4| 1| 1| 70.0|2020-08-31| null| 70.00|
 5| 7| 1| 80.5|2020-08-31| 1| 74.87|
 5| 9| 1| 40.5|2020-08-31| 1| 37.67|
 6| 8| 1| 84.0|2020-08-31| 1| 78.12|
 7| 4| 1| 95.0|2020-08-31| 1| 88.35|
 8| 5| 1| 100.5|2020-08-31| 1| 93.47|
 9| 7| 1| 78.5|2020-08-31| null| 78.50|
 9| 10| 1| 29.99|2020-08-31| null| 29.99|
 10| 6| 1| 124.5|2020-08-31| 1| 115.79|
 11| 3| 1| 83.0|2020-08-31| 1| 77.19|
 12| 1| 1| 70.0|2020-08-31| 1| 65.10|
 13| 4| 1| 95.0|2020-08-31| 1| 88.35|
 14| 9| 1| 42.5|2020-08-31| 1| 39.53|
 15| 6| 1| 115.0|2020-08-31| 1| 106.95|
 15| 2| 1| 78.5|2020-08-31| 1| 73.01|
 17| 8| 1| 84.0|2020-08-31| 1| 78.12|
+----------+---------+---------------+-------+----------+-----------+-----------+
only showing top 20 rows

In [0]:
# Para dejar la tabla como el df_facturas_mes_actual más el campo "importe_dto", simplemente eliminamos "count_contratos" y "flg_mes_ant".

df_facturas_mes_actual_descuento = df_facturas_mes_actual_descuento.drop("count_contratos", "flg_mes_ant")

df_facturas_mes_actual_descuento.show(10)

+----------+---------+-------+----------+-----------+
id_cliente|id_oferta|importe| fecha|importe_dto|
+----------+---------+-------+----------+-----------+
 1| 6| 124.5|2020-08-31| 115.79|
 1| 6| 118.6|2020-08-31| 110.30|
 2| 4| 95.0|2020-08-31| 88.35|
 3| 5| 102.5|2020-08-31| 95.33|
 4| 1| 70.0|2020-08-31| 70.00|
 5| 7| 80.5|2020-08-31| 74.87|
 5| 9| 40.5|2020-08-31| 37.67|
 6| 8| 84.0|2020-08-31| 78.12|
 7| 4| 95.0|2020-08-31| 88.35|
 8| 5| 100.5|2020-08-31| 93.47|
+----------+---------+-------+----------+-----------+
only showing top 10 rows

**3. Por problemas de saturación en la red debido al incremento en el uso de datos por el confinamiento debido a la COVID-19, se decide limitar el uso de datos este mes subiendo la tarifa de todas las ofertas que incluyan "datos ilimitados" en un 15%. Obtener un DF con las mismas columnas que el DF de facturas del mes actual y a mayores la columna "importe_dto"**.


Interpretamos que cuando se indica incluir la columna "importe_dto", ésta debe contener el nuevo importe con el incremento del 15 % que se comenta.
En este caso necesitaremos las tablas de df_facturas_mes_actual y df_ofertas para comprobar que ofertas incluyen datos ilimitados. 

In [0]:
df_ofertas = spark.read.table("df_ofertas")
df_facturas_mes_actual = spark.read.table("df_facturas_mes_actual")

# Vamos a añadir a la tabla de df_facturas_mes_actual la información de df_ofertas, realizando un left join por el id_oferta, pero no arrastraremos la columna "importe" de df_ofertas para no confundirlas con la de facturas.

df_facturas_mes_actual_with_ofertas = df_facturas_mes_actual.join(df_ofertas.select("id_oferta","descripcion"),["id_oferta"],"left").sort("id_cliente")

df_facturas_mes_actual_with_ofertas.show(n= 10, truncate=False)

+---------+----------+-------+----------+------------------------------------------------------------------------------------+
id_oferta|id_cliente|importe|fecha |descripcion |
+---------+----------+-------+----------+------------------------------------------------------------------------------------+
6 |1 |124.5 |2020-08-31|Fibra Optica 600MB + Fijo ilimitado + Movil llamadas y datos ilimitados + Television|
6 |1 |118.6 |2020-08-31|Fibra Optica 600MB + Fijo ilimitado + Movil llamadas y datos ilimitados + Television|
4 |2 |95.0 |2020-08-31|Fibra Optica 600MB + Movil llamadas y datos ilimitados + Television |
5 |3 |102.5 |2020-08-31|Fibra Optica 300MB + Fijo ilimitado + Movil llamadas y datos ilimitados |
1 |4 |70.0 |2020-08-31|Fibra Optica 300MB + Fijo ilimitado + Television |
7 |5 |80.5 |2020-08-31|Fibra Optica 300MB + Movil llamadas ilimitadas + Television |
9 |5 |40.5 |2020-08-31|Fijo llamadas ilimitadas + Movil llamadas y datos ilimitados |
8 |6 |84.0 |2020-08-31|Fibra Optica 300MB + Movil datos ilimitados + Television |
4 |7 |95.0 |2020-08-31|Fibra Optica 600MB + Movil llamadas y datos ilimitados + Television |
5 |8 |100.5 |2020-08-31|Fibra Optica 300MB + Fijo ilimitado + Movil llamadas y datos ilimitados |
+---------+----------+-------+----------+------------------------------------------------------------------------------------+
only showing top 10 rows

In [0]:
# Ahora crearemos una nueva columna "importe_dto" con el incremento del 15% en función de si la oferta incluye "datos ilimitados" o no. 

df_facturas_mes_actual_with_ofertas_new_price = df_facturas_mes_actual_with_ofertas.withColumn("importe_dto", when(col("descripcion").contains("datos ilimitados"),
                                                                                                              (col("importe")*1.15).cast(DecimalType(8,2))).otherwise(col("importe")))

df_facturas_mes_actual_with_ofertas_new_price.show(n=10, truncate = False)            

+---------+----------+-------+----------+------------------------------------------------------------------------------------+-----------+
id_oferta|id_cliente|importe|fecha |descripcion |importe_dto|
+---------+----------+-------+----------+------------------------------------------------------------------------------------+-----------+
6 |1 |124.5 |2020-08-31|Fibra Optica 600MB + Fijo ilimitado + Movil llamadas y datos ilimitados + Television|143.17 |
6 |1 |118.6 |2020-08-31|Fibra Optica 600MB + Fijo ilimitado + Movil llamadas y datos ilimitados + Television|136.39 |
4 |2 |95.0 |2020-08-31|Fibra Optica 600MB + Movil llamadas y datos ilimitados + Television |109.25 |
5 |3 |102.5 |2020-08-31|Fibra Optica 300MB + Fijo ilimitado + Movil llamadas y datos ilimitados |117.87 |
1 |4 |70.0 |2020-08-31|Fibra Optica 300MB + Fijo ilimitado + Television |70.0 |
7 |5 |80.5 |2020-08-31|Fibra Optica 300MB + Movil llamadas ilimitadas + Television |80.5 |
9 |5 |40.5 |2020-08-31|Fijo llamadas ilimitadas + Movil llamadas y datos ilimitados |46.57 |
8 |6 |84.0 |2020-08-31|Fibra Optica 300MB + Movil datos ilimitados + Television |96.6 |
4 |7 |95.0 |2020-08-31|Fibra Optica 600MB + Movil llamadas y datos ilimitados + Television |109.25 |
5 |8 |100.5 |2020-08-31|Fibra Optica 300MB + Fijo ilimitado + Movil llamadas y datos ilimitados |115.57 |
+---------+----------+-------+----------+------------------------------------------------------------------------------------+-----------+
only showing top 10 rows

In [0]:
# Como el enunciado nos pide un DF con las mismas columnas que el DF de facturas del mes actual más la nueva columna que hemos creado, sólo nos quedará eliminar la columna descripcion, o bien, si además queremos 
# cambiar la posición de cada columna, podemos utilizar el comando select y seleccionar sólo las columnas que necesitemos en el orden que queramos que aparezcan.

df_facturas_mes_actual_nuevo_precio = df_facturas_mes_actual_with_ofertas_new_price.select("id_cliente", "id_oferta", "importe", "fecha", "importe_dto")

# Para simplemente eliminar la columna descripcion:
# df_facturas_mes_actual_and_new_price = df_facturas_mes_actual_with_ofertas_new_price.drop("descripcion")


df_facturas_mes_actual_nuevo_precio.show(5)
df_facturas_mes_actual.orderBy("id_cliente").show(5)

+----------+---------+-------+----------+-----------+
id_cliente|id_oferta|importe| fecha|importe_dto|
+----------+---------+-------+----------+-----------+
 1| 6| 124.5|2020-08-31| 143.17|
 1| 6| 118.6|2020-08-31| 136.39|
 2| 4| 95.0|2020-08-31| 109.25|
 3| 5| 102.5|2020-08-31| 117.87|
 4| 1| 70.0|2020-08-31| 70.0|
+----------+---------+-------+----------+-----------+
only showing top 5 rows

+----------+---------+-------+----------+
id_cliente|id_oferta|importe| fecha|
+----------+---------+-------+----------+
 1| 6| 118.6|2020-08-31|
 1| 6| 124.5|2020-08-31|
 2| 4| 95.0|2020-08-31|
 3| 5| 102.5|2020-08-31|
 4| 1| 70.0|2020-08-31|
+----------+---------+-------+----------+
only showing top 5 rows

**4. Crear una nueva variable "grupo_edad" que agrupe a los clientes, tanto del mes_actual como del mes_anterior en 4 rangos según su edad asignando valores del 1 - 4 segun el rango al que pertenezcan (18-25 (1), 26-40 (2), 41-65 (3), >65 (4)). Obtener una tabla resumen que extraiga para cada para cada uno de los 4 grupos identificados la MEDIA de consumo de datos, sms enviados, minutos_movil, minutos_fijo, con todos los campos casteados a 2 decimales y ordenar el DF por grupo_edad ascendente. Extraer conclusiones.**

In [0]:
# En primer lugar, tenemos que tener en cuenta que hay que crear la nueva variable tanto para el dataframe del mes anterior (df_facturas_mes_ant) como en el del mes actual (df_facturas_mes_actual), puesto que aplica a ambos.
# Sin embargo, la columna o campo "edad" no existe en estos dataframes, sino que viene en el dataframe que contiene información de los clientes (df_clientes). Por tanto, necesitamos agregar a las dos tablas esta información mediante el uso del comando join.
# Así mismo, para obtener la media de consumo de datos, sms enviados, etc, necesitaremos utilizar la tabla de df_consumos_diarios.


df_facturas_mes_actual = spark.read.table("df_facturas_mes_actual")
df_facturas_mes_ant = spark.read.table("df_facturas_mes_ant")
df_clientes = spark.read.table("df_clientes")
df_consumos_diarios = spark.read.table("df_consumos_diarios")

# Realizamos un "left join" porque queremos agregar información a los dataframes de facturas, manteniendo todos los registros (filas) y columnas del mismo dataframe.
df_facturas_mes_ant_y_edad = df_facturas_mes_ant.join(df_clientes.select("id_cliente","edad"),["id_cliente"],"left").orderBy("id_cliente")
df_facturas_mes_actual_y_edad = df_facturas_mes_actual.join(df_clientes.select("id_cliente","edad"),["id_cliente"],"left").orderBy("id_cliente")


df_facturas_mes_ant_y_edad.show(5)
df_facturas_mes_actual_y_edad.show(5)

+----------+---------+-------+----------+----+
id_cliente|id_oferta|importe| fecha|edad|
+----------+---------+-------+----------+----+
 1| 9| 36.5|2020-07-31| 26|
 1| 6| 124.5|2020-07-31| 26|
 1| 6| 118.6|2020-07-31| 26|
 2| 4| 95.0|2020-07-31| 58|
 3| 5| 102.5|2020-07-31| 68|
+----------+---------+-------+----------+----+
only showing top 5 rows

+----------+---------+-------+----------+----+
id_cliente|id_oferta|importe| fecha|edad|
+----------+---------+-------+----------+----+
 1| 6| 124.5|2020-08-31| 26|
 1| 6| 118.6|2020-08-31| 26|
 2| 4| 95.0|2020-08-31| 58|
 3| 5| 102.5|2020-08-31| 68|
 4| 1| 70.0|2020-08-31| 32|
+----------+---------+-------+----------+----+
only showing top 5 rows

In [0]:
# Antes de continuar, debemos darnos cuenta, de cara a buscar soluciones óptimas, que la tabla de consumos sólo contiene información del mes actual, por tanto no tiene sentido que realicemos un join entre las tablas de facturas para obtener la tabla resumen de los distintos consumos por grupo de edad.

df_consumos_diarios.agg({'fecha': 'max'}).show()

df_consumos_diarios.agg({'fecha': 'min'}).show()



+----------+
max(fecha)|
+----------+
2020-08-31|
+----------+

+----------+
min(fecha)|
+----------+
2020-08-01|
+----------+

In [0]:
# Procederemos por tanto  a crear la nueva columna "grupo_edad" para ambas tablas, y luego obtendremos la tabla resumen utilizando la tabla que obtengamos del mes_actual y df_consumos_diarios. 

df_facturas_mes_ant_grupoedad =  df_facturas_mes_ant_y_edad.withColumn("grupo_edad", when(col("edad") < 26, lit(1))
                                         .when(col("edad").between(26,40),lit(2))
                                         .when(col("edad").between(41,65),lit(3))
                                         .otherwise(lit(4)))

df_facturas_mes_actual_grupoedad =  df_facturas_mes_actual_y_edad.withColumn("grupo_edad", when(col("edad") < 26, lit(1))
                                         .when(col("edad").between(26,40),lit(2))
                                         .when(col("edad").between(41,65),lit(3))
                                         .otherwise(lit(4)))


df_facturas_mes_ant_grupoedad.show(5)

df_facturas_mes_actual_grupoedad.show(5)

+----------+---------+-------+----------+----+----------+
id_cliente|id_oferta|importe| fecha|edad|grupo_edad|
+----------+---------+-------+----------+----+----------+
 1| 9| 36.5|2020-07-31| 26| 2|
 1| 6| 124.5|2020-07-31| 26| 2|
 1| 6| 118.6|2020-07-31| 26| 2|
 2| 4| 95.0|2020-07-31| 58| 3|
 3| 5| 102.5|2020-07-31| 68| 4|
+----------+---------+-------+----------+----+----------+
only showing top 5 rows

+----------+---------+-------+----------+----+----------+
id_cliente|id_oferta|importe| fecha|edad|grupo_edad|
+----------+---------+-------+----------+----+----------+
 1| 6| 118.6|2020-08-31| 26| 2|
 1| 6| 124.5|2020-08-31| 26| 2|
 2| 4| 95.0|2020-08-31| 58| 3|
 3| 5| 102.5|2020-08-31| 68| 4|
 4| 1| 70.0|2020-08-31| 32| 2|
+----------+---------+-------+----------+----+----------+
only showing top 5 rows

In [0]:
# Ahora a este nuevo DF de facturas del mes actual deberemos agregar el df de consumos diarios, y dispondremos de toda la información para construir la tabla resumen. Eliminaremos duplicados utilizando como claves todos los campos de la tabla de consumos, para asegurarnos que no hay consumos duplicados. Además ambas tabla contienen un columna llamada fecha pero con datos diferentes. En La tabla de facturas se indica la fecha mensual, mientras que la de consumos, la fecha diaria, por la tanto las renombraremos para evitar confusión.

# Si el left join lo realizaramos sobre la tabla de facturas del mes anterior, se agregarían sólo los consumos actuales de los clientes que ya existían en el mes anterior, no dispondríamos de los consumos de los clientes nuevos.

df_facturas_grupoedad_consumos = df_facturas_mes_actual_grupoedad.withColumnRenamed("fecha","fecha mes").join(df_consumos_diarios.withColumnRenamed("fecha","fecha diaria"), ["id_cliente"], "left") \
                                                      .dropDuplicates(["id_cliente", "consumo_datos_MB","sms_enviados","minutos_llamadas_movil","minutos_llamadas_fijo","fecha diaria"])

df_facturas_grupoedad_consumos.show(10)

+----------+---------+-------+----------+----+----------+----------------+------------+----------------------+---------------------+------------+
id_cliente|id_oferta|importe| fecha mes|edad|grupo_edad|consumo_datos_MB|sms_enviados|minutos_llamadas_movil|minutos_llamadas_fijo|fecha diaria|
+----------+---------+-------+----------+----+----------+----------------+------------+----------------------+---------------------+------------+
 1| 6| 124.5|2020-08-31| 26| 2| 1627| 0| 20| 0| 2020-08-06|
 1| 6| 124.5|2020-08-31| 26| 2| 760| 0| 12| 0| 2020-08-08|
 1| 6| 124.5|2020-08-31| 26| 2| 760| 0| 25| 0| 2020-08-12|
 1| 6| 124.5|2020-08-31| 26| 2| 603| 0| 41| 0| 2020-08-13|
 1| 6| 124.5|2020-08-31| 26| 2| 734| 0| 19| 0| 2020-08-25|
 1| 6| 124.5|2020-08-31| 26| 2| 1729| 0| 11| 0| 2020-08-26|
 1| 6| 124.5|2020-08-31| 26| 2| 302| 0| 29| 0| 2020-08-29|
 1| 6| 124.5|2020-08-31| 26| 2| 760| 1| 31| 0| 2020-08-16|
 1| 6| 124.5|2020-08-31| 26| 2| 1242| 1| 40| 0| 2020-08-22|
 1| 6| 124.5|2020-08-31| 26| 2| 829| 2| 46| 0| 2020-08-10|
+----------+---------+-------+----------+----+----------+----------------+------------+----------------------+---------------------+------------+
only showing top 10 rows

In [0]:
# Comprobamos que no se han duplicado registros de consumos.

print(str(df_consumos_diarios.count()))
print(str(df_facturas_grupoedad_consumos.count()))

837
837

In [0]:
# Ejecutamos el comando groupBy por la columna grupo_edad, que nos separará los registros en tantos grupos como valores distintos contenga dicha columna (es decir 4), una vez agrupados, podremos realizar operaciones sobre ellos y crear nuevas columnas en las que volcar el resultado de dichas operaciones. En este caso, calculamos la suma de cada grupo en cada uno de los distintos consumos, el número de clientes de cada grupo mediante el comando countDistinct, y con esta información podemos calcular la media por persona de cada tipo de consumo en cada grupo de edad. Este resultado lo casteamos a 10 enteros y 2 decimales, y ordenamos la tabla final por grupo_edad (si no especificamos, el valor por defecto es ascendente).

df_facturas_grupoedad_consumos_resumen = df_facturas_grupoedad_consumos \
                                                         .groupBy("grupo_edad").agg(sum("consumo_datos_MB").alias("consumo_datos_total"),
                                                                                    sum("sms_enviados").alias("sms_enviados_total"),
                                                                                    sum("minutos_llamadas_movil").alias("minutos_llamadas_movil_total"),
                                                                                    sum("minutos_llamadas_fijo").alias("minutos_llamadas_fijo_total"),
                                                                                    countDistinct("id_cliente").alias("n_clientes")) \
                                                         .withColumn("consumo_medio_datos", (col("consumo_datos_total") / col("n_clientes")).cast(DecimalType(10,2)))\
                                                         .withColumn("consumo_medio_sms", (col("sms_enviados_total") / col("n_clientes")).cast(DecimalType(10,2)))\
                                                         .withColumn("consumo_medio_llamadas_movil", (col("minutos_llamadas_movil_total") / col("n_clientes")).cast(DecimalType(10,2)))\
                                                         .withColumn("consumo_medio_llamadas_fijo", (col("minutos_llamadas_fijo_total") / col("n_clientes")).cast(DecimalType(10,2)))\
                                                         .orderBy("grupo_edad")


df_facturas_grupoedad_consumos_resumen.show()

+----------+-------------------+------------------+----------------------------+---------------------------+----------+-------------------+-----------------+----------------------------+---------------------------+
grupo_edad|consumo_datos_total|sms_enviados_total|minutos_llamadas_movil_total|minutos_llamadas_fijo_total|n_clientes|consumo_medio_datos|consumo_medio_sms|consumo_medio_llamadas_movil|consumo_medio_llamadas_fijo|
+----------+-------------------+------------------+----------------------------+---------------------------+----------+-------------------+-----------------+----------------------------+---------------------------+
 1| 207427| 83| 2940| 1684| 5| 41485.40| 16.60| 588.00| 336.80|
 2| 268556| 280| 8649| 5396| 8| 33569.50| 35.00| 1081.13| 674.50|
 3| 132049| 1466| 9050| 7375| 9| 14672.11| 162.89| 1005.56| 819.44|
 4| 26535| 955| 2804| 5542| 5| 5307.00| 191.00| 560.80| 1108.40|
+----------+-------------------+------------------+----------------------------+---------------------------+----------+-------------------+-----------------+----------------------------+---------------------------+


###### Conclusiones


+ El grupo de edad **1 (18-25 años)** posee el mayor consumo medio de datos, y a su vez el más bajo de sms y de llamadas a fijos. Estos datos denotan que para la generación más joven, los sms y teléfonos fijos son herramientas obsoletas o tendentes a su desaparición. Este grupo de edad demuestra que prioriza el uso de internet en su forma de comunicarse, aunque tampoco tiene el consumo más bajo en llamadas por móvil, probablemente limitado por su bajo poder adquisitivo en comparación al resto de grupos.

+ El grupo de edad __2 (26-40 años)__ sigue una línea similar al anterior, muestra el segundo mayor consumo medio de datos, y los segundos más bajos de sms y llamadas por fijo. Este grupo tambíén consume preferiblemente internet, aunque el de llamadas por móvil es también muy importante.

+ El grupo de edad **3 (41-65 años)** ya denota un descenso destacado en el consumo de datos, posiblemente relacionado a, generalmente, un menor conocimiento del uso de internet y las nuevas tecnologías. Este grupo de edad ya muestra una tendencia a un uso de formas de comunicarse más tradicionales como los sms y las llamadas por fijo, y es el mayor consumidor de llamadas por móvil.

+ El grupo de edad **4 (> 65 años)** es, con bastante diferencia, el que ostenta el menor consumo medio de datos, y a su vez, el mayor consumidor de sms y llamadas por fijo, probablemente por las mismas razones que el grupo de edad 3. Su consumo en llamadas por móvil es el más limitado.


En líneas generales, podemos intuir que el consumo medio de datos tenderá a crecer, pues va ligado al conocimiento de nuevas tecnologías e internet, propio de generaciones más jóvenes.
El consumo de sms y llamadas a fijos irá progresivamente desapareciendo y, en cambio, las llamadas a móviles se mantendrían estables, pues parecen ir más vinculadas al poder adquisitivo del que dispone el cliente. 



**5. Se quiere realizar un estudio por Sexo para analizar si son las mujeres o los hombres quiénes consumen más datos durante el fin de semana y hacen más llamadas desde el móvil. Para ello se deberá, sin ayuda de un calendario, extraer el día de la semana al que corresponde cada una de las fechas del mes de
Agosto para saber cuáles son fin de semana (se consideran días de fin de semana el viernes, sábado y domingo). </br> El DF a obtener deberá tener 2 registros con las columnas: sexo, total_mins_movil_finde, total_datos_moviles_finde. </br> Extraer conclusiones tras presentar el DF resultante.**

In [0]:
# Para obtener el dato del sexo del cliente necesitaremos la tabla df_clientes, y para hacer el estudio del consumo en fin de semana, la tabla df_consumos_diarios.

df_clientes = spark.read.table("df_clientes")
df_consumos_diarios = spark.read.table("df_consumos_diarios")

df_clientes.show(10)
df_consumos_diarios.show(10)

+----------+-----------------+----+----+-----------+--------+
id_cliente| nombre|edad|sexo| provincia| pais|
+----------+-----------------+----+----+-----------+--------+
 10| Pablo Lopez| 40| M| Sevilla| España|
 11| Luis Rodrigues| 20| M| Bogota|Colombia|
 12| Alvaro Monroy| 75| M| Monterrey| Mexico|
 13| Victor Ruiz| 53| M| Valladolid| España|
 14| Melisa Aguado| 30| F| Bogota|Colombia|
 15|Cristian Cuadrado| 52| M|Guadalajara| Mexico|
 25| Ines Barcero| 29| F|Guadalajara| Mexico|
 26| Celia Castro| 47| F| Barcelona| España|
 27| Roberta Varado| 64| F| Cali|Colombia|
 28| Walter Ramos| 54| M| Cali|Colombia|
+----------+-----------------+----+----+-----------+--------+
only showing top 10 rows

+----------+----------------+------------+----------------------+---------------------+----------+
id_cliente|consumo_datos_MB|sms_enviados|minutos_llamadas_movil|minutos_llamadas_fijo| fecha|
+----------+----------------+------------+----------------------+---------------------+----------+
 27| 249| 6| 24| 24|2020-08-16|
 27| 242| 1| 31| 23|2020-08-17|
 27| 237| 15| 15| 57|2020-08-18|
 27| 264| 6| 7| 54|2020-08-19|
 27| 255| 1| 41| 19|2020-08-20|
 27| 251| 13| 7| 44|2020-08-21|
 27| 281| 2| 30| 34|2020-08-22|
 27| 338| 3| 23| 43|2020-08-23|
 27| 201| 13| 8| 57|2020-08-24|
 27| 277| 11| 13| 38|2020-08-25|
+----------+----------------+------------+----------------------+---------------------+----------+
only showing top 10 rows


Para obtener el día de la semana utilizaremos la función **dayofweek()**, perteneciente a la librería  pyspark.sql.functions que ya hemos importado al comienzo del notebook.
Esta función asigna un número entero a cada día de la semana, en un rango lógicamente del 1 al 7, siendo el 1 para Domingo y 7 para Sábado.


https://sparkbyexamples.com/pyspark/pyspark-sql-date-and-timestamp-functions/#pyspark-sql-date-functions

In [0]:
# Si quisieramos obtener el día de la semana al que pertenece cada fecha, lo podríamos hacer de la siguiente forma:

df_consumos_diarios_dia_semana = df_consumos_diarios.withColumn("día semana", dayofweek("fecha")) \
                                                    .withColumn("día semana",  when(col("día semana") == 1, lit("Domingo"))
                                                               .when(col("día semana") == 2, lit("Lunes"))
                                                               .when(col("día semana") == 3, lit("Martes"))
                                                               .when(col("día semana") == 4, lit("Miércoles"))
                                                               .when(col("día semana") == 5, lit("Jueves"))
                                                               .when(col("día semana") == 6, lit("Viernes"))
                                                               .otherwise(lit("Sábado")))

df_consumos_diarios_dia_semana.show(10)

+----------+----------------+------------+----------------------+---------------------+----------+----------+
id_cliente|consumo_datos_MB|sms_enviados|minutos_llamadas_movil|minutos_llamadas_fijo| fecha|día semana|
+----------+----------------+------------+----------------------+---------------------+----------+----------+
 27| 249| 6| 24| 24|2020-08-16| Domingo|
 27| 242| 1| 31| 23|2020-08-17| Lunes|
 27| 237| 15| 15| 57|2020-08-18| Martes|
 27| 264| 6| 7| 54|2020-08-19| Miércoles|
 27| 255| 1| 41| 19|2020-08-20| Jueves|
 27| 251| 13| 7| 44|2020-08-21| Viernes|
 27| 281| 2| 30| 34|2020-08-22| Sábado|
 27| 338| 3| 23| 43|2020-08-23| Domingo|
 27| 201| 13| 8| 57|2020-08-24| Lunes|
 27| 277| 11| 13| 38|2020-08-25| Martes|
+----------+----------------+------------+----------------------+---------------------+----------+----------+
only showing top 10 rows

In [0]:
# Pero como sólonos interesa clasificar las fechas que pertenecen al fin de semana, creamos la columna "día semana" apartir de la columna "fecha" de consumos diarios, y luego la modificaremos para que nos muestre un string indicando si el día pertenece al fin de semana, en lugar del número entero que asigna por defecto.

df_consumos_diarios_dia_semana = df_consumos_diarios.withColumn("día semana", dayofweek("fecha")) \
                                                    .withColumn("día semana",  when((col("día semana") == 1) |(col("día semana") == 6) | (col("día semana") == 7), lit("Fin de Semana"))
                                                                .otherwise(lit("Resto Semana")))

df_consumos_diarios_dia_semana.show()

+----------+----------------+------------+----------------------+---------------------+----------+-------------+
id_cliente|consumo_datos_MB|sms_enviados|minutos_llamadas_movil|minutos_llamadas_fijo| fecha| día semana|
+----------+----------------+------------+----------------------+---------------------+----------+-------------+
 27| 249| 6| 24| 24|2020-08-16|Fin de Semana|
 27| 242| 1| 31| 23|2020-08-17| Resto Semana|
 27| 237| 15| 15| 57|2020-08-18| Resto Semana|
 27| 264| 6| 7| 54|2020-08-19| Resto Semana|
 27| 255| 1| 41| 19|2020-08-20| Resto Semana|
 27| 251| 13| 7| 44|2020-08-21|Fin de Semana|
 27| 281| 2| 30| 34|2020-08-22|Fin de Semana|
 27| 338| 3| 23| 43|2020-08-23|Fin de Semana|
 27| 201| 13| 8| 57|2020-08-24| Resto Semana|
 27| 277| 11| 13| 38|2020-08-25| Resto Semana|
 27| 299| 12| 28| 19|2020-08-26| Resto Semana|
 27| 139| 1| 22| 47|2020-08-27| Resto Semana|
 27| 267| 10| 16| 40|2020-08-28|Fin de Semana|
 27| 178| 8| 19| 27|2020-08-29|Fin de Semana|
 27| 177| 17| 9| 45|2020-08-30|Fin de Semana|
 27| 87| 12| 11| 18|2020-08-31| Resto Semana|
 28| 1124| 14| 5| 12|2020-08-01|Fin de Semana|
 28| 1044| 7| 7| 7|2020-08-02|Fin de Semana|
 28| 755| 13| 6| 6|2020-08-03| Resto Semana|
 28| 1533| 18| 7| 2|2020-08-04| Resto Semana|
+----------+----------------+------------+----------------------+---------------------+----------+-------------+
only showing top 20 rows

In [0]:
# Ahora, a esta nueva tabla de consumos, le añadiremos la información del sexo de cada cliente, utilizando para ello la tabla df_clientes

df_consumos_diarios_dia_semana_sexo = df_consumos_diarios_dia_semana.join(df_clientes.select("id_cliente","sexo"), ["id_cliente"], "left")

df_consumos_diarios_dia_semana_sexo.show()

+----------+----------------+------------+----------------------+---------------------+----------+-------------+----+
id_cliente|consumo_datos_MB|sms_enviados|minutos_llamadas_movil|minutos_llamadas_fijo| fecha| día semana|sexo|
+----------+----------------+------------+----------------------+---------------------+----------+-------------+----+
 27| 249| 6| 24| 24|2020-08-16|Fin de Semana| F|
 27| 242| 1| 31| 23|2020-08-17| Resto Semana| F|
 27| 237| 15| 15| 57|2020-08-18| Resto Semana| F|
 27| 264| 6| 7| 54|2020-08-19| Resto Semana| F|
 27| 255| 1| 41| 19|2020-08-20| Resto Semana| F|
 27| 251| 13| 7| 44|2020-08-21|Fin de Semana| F|
 27| 281| 2| 30| 34|2020-08-22|Fin de Semana| F|
 27| 338| 3| 23| 43|2020-08-23|Fin de Semana| F|
 27| 201| 13| 8| 57|2020-08-24| Resto Semana| F|
 27| 277| 11| 13| 38|2020-08-25| Resto Semana| F|
 27| 299| 12| 28| 19|2020-08-26| Resto Semana| F|
 27| 139| 1| 22| 47|2020-08-27| Resto Semana| F|
 27| 267| 10| 16| 40|2020-08-28|Fin de Semana| F|
 27| 178| 8| 19| 27|2020-08-29|Fin de Semana| F|
 27| 177| 17| 9| 45|2020-08-30|Fin de Semana| F|
 27| 87| 12| 11| 18|2020-08-31| Resto Semana| F|
 28| 1124| 14| 5| 12|2020-08-01|Fin de Semana| M|
 28| 1044| 7| 7| 7|2020-08-02|Fin de Semana| M|
 28| 755| 13| 6| 6|2020-08-03| Resto Semana| M|
 28| 1533| 18| 7| 2|2020-08-04| Resto Semana| M|
+----------+----------------+------------+----------------------+---------------------+----------+-------------+----+
only showing top 20 rows

In [0]:
# A partir de la tabla anterior crearemos el DF final que deberá contener 2 registros con las columnas: sexo, total_mins_movil_finde, total_datos_moviles_finde.
# Para ello filtraremos los registros que cumplan la condición de pertenecer al fin de semana, los agruparemos en funcion del sexo, y agregaremos 2 columnas que contendran la suma de minutos de llamadas por movil y de consumo de datos respectivamente.

df_estudio_finde_sexo  = df_consumos_diarios_dia_semana_sexo.filter(col("día semana").contains("Fin de Semana")) \
                                                            .groupBy("sexo").agg(sum("minutos_llamadas_movil").alias("total_mins_movil_finde"),
                                                                                 sum("consumo_datos_MB").alias("total_datos_moviles_finde"))

df_estudio_finde_sexo.show()

+----+----------------------+-------------------------+
sexo|total_mins_movil_finde|total_datos_moviles_finde|
+----+----------------------+-------------------------+
 F| 7222| 148009|
 M| 3459| 139267|
+----+----------------------+-------------------------+


###### Conclusiones

Podemos confirmar que, tanto para llamadas por móvil como para uso de datos, las mujeres muestran unos consumos mayores que los hombres en el fin de semana.
En el caso de las llamadas, la diferencia es muy notable, pues las mujeres consumen más del doble de minutos que consumen los hombres.
En el caso de los datos, la diferencia no es tan significativa.

**6. Obtener un DF que contenga 4 registros, que serán el cliente de cada grupo edad que más datos móviles ha consumido durante los 15 primeros días del mes de Agosto (día 15 incluido en el cálculo). El DF deberá contener las columnas: nombre, edad, grupo_edad, datos_moviles_total_15, max_sms_enviados_15** </br> (max_sms_enviados_15 contiene el máximo de sms enviados en un día por el cliente con más datos_moviles consumidos de cada grupo_edad durante esos 15 primeros días del mes). </br>
**Extraer conclusiones en cuanto a datos consumidos y sms enviados por cada grupo_edad.**

In [0]:
# En este caso, necesitaremos las tabla de consumos diarios y la tabla de clientes, para añadirle a la primera tabla el nombre del cliente y la columna edad para poder calcular la columna grupo edad. 
# Acto seguido, filtraremos por los días 1-15 del mes de Agosto, filtraremos por grupo edad y buscaremos la mejor manera de agrupar los id_cliente de cara a calcular suma del consumo de datos de cada cliente en esos días y el máximo de sms enviados en un día concreto.

df_consumos_diarios = spark.read.table("df_consumos_diarios")
df_clientes = spark.read.table("df_clientes")

df_consumos_diarios.show(3)
df_clientes.show(3)

+----------+----------------+------------+----------------------+---------------------+----------+
id_cliente|consumo_datos_MB|sms_enviados|minutos_llamadas_movil|minutos_llamadas_fijo| fecha|
+----------+----------------+------------+----------------------+---------------------+----------+
 27| 249| 6| 24| 24|2020-08-16|
 27| 242| 1| 31| 23|2020-08-17|
 27| 237| 15| 15| 57|2020-08-18|
+----------+----------------+------------+----------------------+---------------------+----------+
only showing top 3 rows

+----------+--------------+----+----+---------+--------+
id_cliente| nombre|edad|sexo|provincia| pais|
+----------+--------------+----+----+---------+--------+
 10| Pablo Lopez| 40| M| Sevilla| España|
 11|Luis Rodrigues| 20| M| Bogota|Colombia|
 12| Alvaro Monroy| 75| M|Monterrey| Mexico|
+----------+--------------+----+----+---------+--------+
only showing top 3 rows

In [0]:
# Procedemos a ejecutar un left join sobre "id_cliente" para mantener todos los registros de la tabla de consumos diarios y agregaremos el nombre y la edad (para poder calcular la columna "grupo edad").
# También eliminaremos las columnas "minutos_llamadas_movil" y "minutos_llamadas_fijo" pues no son necesarias para lo que nos pide el enunciado.

df_consumos_diarios_grupoedad_nombre = df_consumos_diarios.join(df_clientes.select("id_cliente", "edad", "nombre"), ["id_cliente"], "left") \
                                                                    .withColumn("grupo_edad", when(col("edad") < 26, lit(1))
                                                                                             .when(col("edad").between(26,40),lit(2))
                                                                                             .when(col("edad").between(41,65),lit(3))
                                                                                             .otherwise(lit(4))) \
                                                                    .drop("minutos_llamadas_movil","minutos_llamadas_fijo")

df_consumos_diarios_grupoedad_nombre.show(3)

+----------+----------------+------------+----------+----+--------------+----------+
id_cliente|consumo_datos_MB|sms_enviados| fecha|edad| nombre|grupo_edad|
+----------+----------------+------------+----------+----+--------------+----------+
 27| 249| 6|2020-08-16| 64|Roberta Varado| 3|
 27| 242| 1|2020-08-17| 64|Roberta Varado| 3|
 27| 237| 15|2020-08-18| 64|Roberta Varado| 3|
+----------+----------------+------------+----------+----+--------------+----------+
only showing top 3 rows

In [0]:
# Ahora filtramos el DF obtenido por las fechas solicitadas, y por cada grupo de edad, para a continuación poder agrupar por cada cliente perteneciente a cada grupo de edad, y obtener el total de sus datos consumidos, 
# así como el máximo de sms enviados en un día por cada cliente.


df_consumos_diarios_grupoedad_1 = df_consumos_diarios_grupoedad_nombre.filter((col("fecha").between("2020-08-01", "2020-08-15"))).filter(col("grupo_edad") == 1) \
                                                                      .groupBy("id_cliente").agg(sum("consumo_datos_MB").alias("datos_moviles_total_15"), 
                                                                                                 max("sms_enviados").alias("max_sms_enviados_15"))
                                                                    
df_consumos_diarios_grupoedad_2 = df_consumos_diarios_grupoedad_nombre.filter((col("fecha").between("2020-08-01", "2020-08-15"))).filter(col("grupo_edad") == 2) \
                                                                      .groupBy("id_cliente").agg(sum("consumo_datos_MB").alias("datos_moviles_total_15"), 
                                                                                                 max("sms_enviados").alias("max_sms_enviados_15"))

df_consumos_diarios_grupoedad_3 = df_consumos_diarios_grupoedad_nombre.filter((col("fecha").between("2020-08-01", "2020-08-15"))).filter(col("grupo_edad") == 3) \
                                                                      .groupBy("id_cliente").agg(sum("consumo_datos_MB").alias("datos_moviles_total_15"), 
                                                                                                 max("sms_enviados").alias("max_sms_enviados_15"))

df_consumos_diarios_grupoedad_4 = df_consumos_diarios_grupoedad_nombre.filter((col("fecha").between("2020-08-01", "2020-08-15"))).filter(col("grupo_edad") == 4) \
                                                                      .groupBy("id_cliente").agg(sum("consumo_datos_MB").alias("datos_moviles_total_15"), 
                                                                                                 max("sms_enviados").alias("max_sms_enviados_15"))
    
    
    
    
df_consumos_diarios_grupoedad_1.show()
df_consumos_diarios_grupoedad_2.show()
df_consumos_diarios_grupoedad_3.show()
df_consumos_diarios_grupoedad_4.show()

+----------+----------------------+-------------------+
id_cliente|datos_moviles_total_15|max_sms_enviados_15|
+----------+----------------------+-------------------+
 22| 27665| 2|
 7| 15402| 3|
 9| 15801| 2|
 17| 25463| 2|
 11| 20388| 2|
+----------+----------------------+-------------------+

+----------+----------------------+-------------------+
id_cliente|datos_moviles_total_15|max_sms_enviados_15|
+----------+----------------------+-------------------+
 29| 23603| 1|
 19| 17343| 4|
 25| 20032| 5|
 1| 14328| 3|
 10| 6919| 7|
 4| 10163| 4|
 14| 13977| 3|
 21| 22222| 3|
+----------+----------------------+-------------------+

+----------+----------------------+-------------------+
id_cliente|datos_moviles_total_15|max_sms_enviados_15|
+----------+----------------------+-------------------+
 26| 6428| 15|
 27| 2770| 14|
 28| 11892| 18|
 5| 4256| 8|
 8| 5258| 15|
 2| 5078| 12|
 13| 9698| 9|
 18| 9468| 7|
 15| 10279| 6|
+----------+----------------------+-------------------+

+----------+----------------------+-------------------+
id_cliente|datos_moviles_total_15|max_sms_enviados_15|
+----------+----------------------+-------------------+
 6| 1506| 22|
 3| 2886| 12|
 12| 1609| 17|
 30| 4657| 16|
 23| 2017| 2|
+----------+----------------------+-------------------+

In [0]:
# Ahora simplemente debemos filtrar por el valor máximo de "datos_moviles_total_15"


# Seleccionamos el valor máximo de la columna "datos_moviles_total_15" y lo recuperamos con el comando collect() especificando la fila y columna 0, para asignarlo a la variable "max_value", la cual
# la utilizaremos para filtrar los registros que cumplan dicha condicion.


max_value = df_consumos_diarios_grupoedad_1.select(max(col('datos_moviles_total_15'))).collect()[0][0]
df_consumos_diarios_grupoedad_1 =  df_consumos_diarios_grupoedad_1.filter( col("datos_moviles_total_15") ==  max_value)

# Hacemos lo propio para el resto de grupos

max_value = df_consumos_diarios_grupoedad_2.select(max(col('datos_moviles_total_15'))).collect()[0][0]
df_consumos_diarios_grupoedad_2 =  df_consumos_diarios_grupoedad_2.filter( col("datos_moviles_total_15") ==  max_value)


max_value = df_consumos_diarios_grupoedad_3.select(max(col('datos_moviles_total_15'))).collect()[0][0]
df_consumos_diarios_grupoedad_3 =  df_consumos_diarios_grupoedad_3.filter( col("datos_moviles_total_15") ==  max_value)


max_value = df_consumos_diarios_grupoedad_4.select(max(col('datos_moviles_total_15'))).collect()[0][0]
df_consumos_diarios_grupoedad_4 =  df_consumos_diarios_grupoedad_4.filter( col("datos_moviles_total_15") ==  max_value)



df_consumos_diarios_grupoedad_1.show()
df_consumos_diarios_grupoedad_2.show()
df_consumos_diarios_grupoedad_3.show()
df_consumos_diarios_grupoedad_4.show()

+----------+----------------------+-------------------+
id_cliente|datos_moviles_total_15|max_sms_enviados_15|
+----------+----------------------+-------------------+
 22| 27665| 2|
+----------+----------------------+-------------------+

+----------+----------------------+-------------------+
id_cliente|datos_moviles_total_15|max_sms_enviados_15|
+----------+----------------------+-------------------+
 29| 23603| 1|
+----------+----------------------+-------------------+

+----------+----------------------+-------------------+
id_cliente|datos_moviles_total_15|max_sms_enviados_15|
+----------+----------------------+-------------------+
 28| 11892| 18|
+----------+----------------------+-------------------+

+----------+----------------------+-------------------+
id_cliente|datos_moviles_total_15|max_sms_enviados_15|
+----------+----------------------+-------------------+
 30| 4657| 16|
+----------+----------------------+-------------------+

In [0]:
# Ya tenemos localizados los clientes que más datos móviles han consumido en la primera quincena, con el máximo de sms que han enviado cada uno.

# Ahora nos queda unir los 4 DF, agregarle las columnas que nos faltan y ordenarlas para visualizarlas como pide el enunciado.
# En este caso, como los 4 DF tienen el mismo número de columnas y siguen el mismo orden, podemos utilizar el comando union(). 
# Si las columnas estuvieran desordenadas, deberíamos utilizar unionByName(), ya que se llaman de la misma forma.

df_grupo_edad_resumen = df_consumos_diarios_grupoedad_1.union(df_consumos_diarios_grupoedad_2).union(df_consumos_diarios_grupoedad_3).union(df_consumos_diarios_grupoedad_4)\
                                                       .join(df_consumos_diarios_grupoedad_nombre.select("id_cliente","nombre", "edad", "grupo_edad"), ["id_cliente"], "left")\
                                                       .dropDuplicates(["id_cliente"])\
                                                       .orderBy("grupo_edad")\
                                                       .select("nombre", "edad", "grupo_edad", "datos_moviles_total_15", "max_sms_enviados_15")
    

display(df_grupo_edad_resumen)

nombre,edad,grupo_edad,datos_moviles_total_15,max_sms_enviados_15
Clara Suarez,21,1,27665,2
Romina Verde,29,2,23603,1
Walter Ramos,54,3,11892,18
Marta Rodrigo,67,4,4657,16



###### Conclusiones

Esta tabla nos viene a confirmar lo que habíamos hallado en el ejercicio 4, y es que los consumos de de datos móviles y sms enviados se van invirtiendo a medida que pasamos de un grupo de edad a otro. Para las generaciones más 
jóvenes el consumo de datos es algo muy asentado, y que muy probablemente se encuentre en estrecha relación con el conocimiento y la preferencia de las nuevas tecnologías y el uso de internet. Y en cambio, el uso de los mensajes de texto es algo muy esporádico o residual para estos grupos de edad.

A medida que avanzamos de grupo de edad nos vamos encontrando la situación contraria, el uso de datos va descendiendo y el de sms demuestra estar aún muy presente entre estas generaciones. 

Podemos interpretar que, cuando hablamos de datos móviles y sms, los grupos 1 y 2 representan un perfil de consumo (basado en el uso de internet), mientras que los grupos 3 y 4 otro perfil bien diferenciado (menos tecnológico
y más cómodo con servicios no tan dinámicos pero más simples, como los sms).

**7. Nos interesa averiguar los minutos de llamadas de los clientes que son nuevos este mes para realizar un estudio del impacto que tendrán en caso de producirse un pico de volumen de llamadas en la red. </br> Obtener un DF que contenga sólo a los clientes nuevos de este mes con 4 columnas: nombre_cliente_nuevo, edad, importe_total_mes_actual, total_minutos. </br> Se ordenará el DF resultante por la columna "total_minutos" que contiene el total de minutos de llamadas fijas y móviles durante este mes para cada cliente, ordenándolos de manera que el primer cliente nuevo que aparezca sea el que menos minutos de llamadas ha realizado en el mes de Agosto.</br>** NOTAS: Si hay solo 2 clientes nuevos, el DF a obtener tendrá sólo 2 registros. </br> Tener presente que a la hora de hacer un join, la clave de unión debe ser única en la tabla de la derecha o de otra manera se multiplicarán los registros resultantes de manera descontrolada.
</br> Tener en cuenta que si un cliente nuevo tiene 2 ofertas contratadas, la columna importe_mes_actual deberá contener la suma de los importes de ambas ofertas.

In [0]:
# Para filtrar los clientes nuevos deberemos utilizar las dos tablas de facturas, y obtener los registros que estan en la tabla del mes actual que no están en la tabla del mes anterior ("leftanti" join).
# Una vez obtenidos los clientes nuevos, deberemos utilizar la tabla de consumos diarios para calcular los consumos solicitados.
# Como también nos piden agregar el nombre y la edad del cliente, deberemos contar también con la tabla de clientes.

df_facturas_mes_ant = spark.read.table("df_facturas_mes_ant")
df_facturas_mes_actual = spark.read.table("df_facturas_mes_actual")
df_consumos_diarios = spark.read.table("df_consumos_diarios")
df_clientes = spark.read.table("df_clientes")


df_facturas_mes_actual_clientes_nuevos = df_facturas_mes_actual.join(df_facturas_mes_ant.select("id_cliente"),["id_cliente"],"leftanti")

df_facturas_mes_actual_clientes_nuevos.show()
print(str(df_facturas_mes_actual_clientes_nuevos.count()))
print(str(df_facturas_mes_actual_clientes_nuevos.dropDuplicates(["id_cliente"]).count()))

+----------+---------+-------+----------+
id_cliente|id_oferta|importe| fecha|
+----------+---------+-------+----------+
 28| 5| 102.5|2020-08-31|
 4| 1| 70.0|2020-08-31|
 9| 7| 78.5|2020-08-31|
 9| 10| 29.99|2020-08-31|
 19| 3| 85.0|2020-08-31|
 21| 6| 124.5|2020-08-31|
+----------+---------+-------+----------+

6
5

In [0]:
# Tenemos  6 facturas que pertenecen a 5 clientes nuevos.
# Como no queremos generar un producto cartesiano, nos aseguraremos de filtrar los id_cliente de los clientes nuevos sin duplicados, antes de añadirle la información de la tabla de consumos.

df_clientes_nuevos = df_facturas_mes_actual_clientes_nuevos.dropDuplicates(["id_cliente"]).select("id_cliente")

df_clientes_nuevos.show()

df_consumos_clientes_nuevos = df_clientes_nuevos.join(df_consumos_diarios.select("id_cliente","minutos_llamadas_movil","minutos_llamadas_fijo"), ["id_cliente"], "left")

df_consumos_clientes_nuevos.show()

+----------+
id_cliente|
+----------+
 19|
 9|
 28|
 4|
 21|
+----------+

+----------+----------------------+---------------------+
id_cliente|minutos_llamadas_movil|minutos_llamadas_fijo|
+----------+----------------------+---------------------+
 19| 13| 9|
 19| 10| 4|
 19| 27| 19|
 19| 17| 16|
 19| 8| 10|
 19| 17| 9|
 19| 21| 29|
 19| 6| 13|
 19| 11| 15|
 19| 11| 18|
 19| 12| 22|
 19| 12| 19|
 19| 7| 9|
 19| 15| 20|
 19| 11| 16|
 19| 10| 7|
 19| 28| 19|
 19| 9| 8|
 19| 20| 8|
 19| 19| 8|
+----------+----------------------+---------------------+
only showing top 20 rows

In [0]:
# Una vez que tenemos los consumos que necesitamos de los clientes nuevos, agruparemos por cliente para calcular el total de minutos consumidos por cada cliente y crearemos la columna "total_minutos", 
# que será la suma de las dos que hemos calculado

df_total_minutos_clientes_nuevos = df_consumos_clientes_nuevos.groupBy("id_cliente").agg(sum("minutos_llamadas_movil").alias("total_minutos_movil"),
                                                                                        sum("minutos_llamadas_fijo").alias("total_minutos_fijo"))\
                                                              .withColumn("total_minutos", col("total_minutos_movil")+col("total_minutos_fijo"))

df_total_minutos_clientes_nuevos.show()

+----------+-------------------+------------------+-------------+
id_cliente|total_minutos_movil|total_minutos_fijo|total_minutos|
+----------+-------------------+------------------+-------------+
 19| 482| 441| 923|
 9| 235| 133| 368|
 28| 268| 260| 528|
 4| 1630| 309| 1939|
 21| 1264| 1102| 2366|
+----------+-------------------+------------------+-------------+

In [0]:
# Para obtener la columna "importe_total_mes_actual" utilizaremos la tabla de facturas del mes actual de clientes nuevos que hemos obtenido al principio de este ejercicio, agruparemos por id_cliente y sumaremos 
# los importe de todas las facturas que pertenezcan a cada cliente

df_total_mes_actual_clientes_nuevos = df_facturas_mes_actual_clientes_nuevos.groupBy("id_cliente").agg(sum("importe").alias("importe_total_mes_actual"))

df_facturas_mes_actual_clientes_nuevos.orderBy("id_cliente").show()

df_total_mes_actual_clientes_nuevos.orderBy("id_cliente").show()

+----------+---------+-------+----------+
id_cliente|id_oferta|importe| fecha|
+----------+---------+-------+----------+
 4| 1| 70.0|2020-08-31|
 9| 7| 78.5|2020-08-31|
 9| 10| 29.99|2020-08-31|
 19| 3| 85.0|2020-08-31|
 21| 6| 124.5|2020-08-31|
 28| 5| 102.5|2020-08-31|
+----------+---------+-------+----------+

+----------+------------------------+
id_cliente|importe_total_mes_actual|
+----------+------------------------+
 4| 70.0|
 9| 108.49|
 19| 85.0|
 21| 124.5|
 28| 102.5|
+----------+------------------------+

In [0]:
# Ahora nos queda añadir a esta tabla la columna de "total_minutos", añadir el nombre y la edad de la tabla df_clientes, renombrar la columna del nombre por "nombre_cliente_nuevo", ordenar las columnas y los registros en función de los minutos totales de forma ascendente (por defecto).

df_clientes_nuevos_resumen = df_total_mes_actual_clientes_nuevos.join(df_total_minutos_clientes_nuevos.select("id_cliente","total_minutos"), ["id_cliente"], "left")\
                                                                .join(df_clientes.select("id_cliente", "nombre", "edad"), ["id_cliente"], "left")\
                                                                .withColumnRenamed("nombre","nombre_cliente_nuevo")\
                                                                .withColumn("importe_total_mes_actual", col("importe_total_mes_actual").cast(DecimalType(10,2)))\
                                                                .select("nombre_cliente_nuevo","edad", "importe_total_mes_actual", "total_minutos")\
                                                                .orderBy("total_minutos")


df_clientes_nuevos_resumen.show()

+--------------------+----+------------------------+-------------+
nombre_cliente_nuevo|edad|importe_total_mes_actual|total_minutos|
+--------------------+----+------------------------+-------------+
 David Cardoso| 24| 108.49| 368|
 Walter Ramos| 54| 102.50| 528|
Juan Carlos Iglesias| 38| 85.00| 923|
 Pedro Conde| 32| 70.00| 1939|
 Fatima Cuevas| 29| 124.50| 2366|
+--------------------+----+------------------------+-------------+

**8. Obtener un DF que contenga, para los clientes que ya existían en el mes anterior y siguen dados de alta este mes, 3 columnas: nombre, edad, n_dias_sin_sms. La última columna se refiere a obtener el número de veces que cada cliente NO ha enviado ningún SMS en el mes de Agosto. Forzar a que el comando "show" muestre 30 valores.** </br> (Si en todo el mes el cliente "A" no envío SMS 3 días en todo el mes, esta columna deberá contener el valor 3. Si hubiera algún cliente que hubiese enviado al menos 1 sms todos los dias del mes, tendrá valor 0 en esta columna).

In [0]:
# Nuevamente vamos a necesitar 4 tablas, las dos de facturas para obtener los clientes actuales que ya existían en el mes anterior, la de consumos para calcular los días sin sms, y la de clientes para añadir el nombre y la edad.

df_facturas_mes_ant = spark.read.table("df_facturas_mes_ant")
df_facturas_mes_actual = spark.read.table("df_facturas_mes_actual")
df_consumos_diarios = spark.read.table("df_consumos_diarios")
df_clientes = spark.read.table("df_clientes")


df_id_clientes_fidelizacion = df_facturas_mes_ant.select("id_cliente").join(df_facturas_mes_actual.select("id_cliente"),["id_cliente"],"inner").dropDuplicates(["id_cliente"]).orderBy("id_cliente")
df_id_clientes_fidelizacion.show()
print("El número TOTAL de clientes que existían en el mes anterior y siguen dados de alta este mes es de: " + str(df_id_clientes_fidelizacion.count()))

+----------+
id_cliente|
+----------+
 1|
 2|
 3|
 5|
 6|
 7|
 8|
 10|
 11|
 12|
 13|
 14|
 15|
 17|
 18|
 22|
 23|
 25|
 26|
 27|
+----------+
only showing top 20 rows

El número TOTAL de clientes que existían en el mes anterior y siguen dados de alta este mes es de: 22

In [0]:
# Ahora que tenemos una tabla con los clientes actuales que existían en el mes anterior, sin duplicados, podemos añadirle los consumos de sms.

df_clientes_fidelizacion_consumos_sms = df_id_clientes_fidelizacion.join(df_consumos_diarios.select("id_cliente", "sms_enviados", "fecha"),["id_cliente"], "left").orderBy("id_cliente", "fecha")

df_clientes_fidelizacion_consumos_sms.show(35)

+----------+------------+----------+
id_cliente|sms_enviados| fecha|
+----------+------------+----------+
 1| 3|2020-08-01|
 1| 0|2020-08-02|
 1| 1|2020-08-03|
 1| 3|2020-08-04|
 1| 1|2020-08-05|
 1| 0|2020-08-06|
 1| 3|2020-08-07|
 1| 0|2020-08-08|
 1| 0|2020-08-09|
 1| 2|2020-08-10|
 1| 3|2020-08-11|
 1| 0|2020-08-12|
 1| 0|2020-08-13|
 1| 0|2020-08-14|
 1| 1|2020-08-15|
 1| 1|2020-08-16|
 1| 2|2020-08-17|
 1| 0|2020-08-18|
 1| 0|2020-08-19|
 1| 2|2020-08-20|
 1| 5|2020-08-21|
 1| 1|2020-08-22|
 1| 0|2020-08-23|
 1| 4|2020-08-24|
 1| 0|2020-08-25|
 1| 0|2020-08-26|
 1| 1|2020-08-27|
 1| 0|2020-08-28|
 1| 0|2020-08-29|
 1| 3|2020-08-30|
 1| 2|2020-08-31|
 2| 7|2020-08-01|
 2| 0|2020-08-02|
 2| 1|2020-08-03|
 2| 1|2020-08-04|
+----------+------------+----------+
only showing top 35 rows

In [0]:
# Vamos a crear una columna que nos ayude a contabilizar los días que un cliente no ha enviados sms. Si el cliente ese día no envió sms, contendrá el valor 1, si el cliento envió, contendrá el valor 0.

df_clientes_fidelizacion_sms_a_0 = df_clientes_fidelizacion_consumos_sms.withColumn("sms_a_0", when(col("sms_enviados") == 0, lit(1)).otherwise(lit(0)))

df_clientes_fidelizacion_sms_a_0.show()

+----------+------------+----------+-------+
id_cliente|sms_enviados| fecha|sms_a_0|
+----------+------------+----------+-------+
 1| 3|2020-08-01| 0|
 1| 0|2020-08-02| 1|
 1| 1|2020-08-03| 0|
 1| 3|2020-08-04| 0|
 1| 1|2020-08-05| 0|
 1| 0|2020-08-06| 1|
 1| 3|2020-08-07| 0|
 1| 0|2020-08-08| 1|
 1| 0|2020-08-09| 1|
 1| 2|2020-08-10| 0|
 1| 3|2020-08-11| 0|
 1| 0|2020-08-12| 1|
 1| 0|2020-08-13| 1|
 1| 0|2020-08-14| 1|
 1| 1|2020-08-15| 0|
 1| 1|2020-08-16| 0|
 1| 2|2020-08-17| 0|
 1| 0|2020-08-18| 1|
 1| 0|2020-08-19| 1|
 1| 2|2020-08-20| 0|
+----------+------------+----------+-------+
only showing top 20 rows

In [0]:
# La idea ahora es agrupar por id_cliente y calcular la suma de este columna "sms_a_0", que nos devolverá el total de días sin enviar sms de cada cliente. 

df_total_dias_sin_sms = df_clientes_fidelizacion_sms_a_0.groupBy("id_cliente")\
                                                        .agg(sum("sms_a_0").alias("n_dias_sin_sms"))

print(str(df_total_dias_sin_sms.count()))

df_total_dias_sin_sms.show(30)

22
+----------+--------------+
id_cliente|n_dias_sin_sms|
+----------+--------------+
 1| 14|
 2| 7|
 3| 3|
 5| 3|
 6| 1|
 7| 19|
 8| 2|
 10| 11|
 11| 21|
 12| 3|
 13| 7|
 14| 20|
 15| 5|
 17| 22|
 18| 7|
 22| 26|
 23| 21|
 25| 13|
 26| 0|
 27| 1|
 29| 23|
 30| 2|
+----------+--------------+

In [0]:
# Ahora simplemente nos queda añadirle las columnas nombre y edad de la tabla df_clientes

df_total_dias_sin_sms_final = df_total_dias_sin_sms.join(df_clientes.select("id_cliente", "nombre", "edad"), ["id_cliente"], "left")\
                                             .select("nombre", "edad", "n_dias_sin_sms")\
                                             .orderBy("n_dias_sin_sms")

df_total_dias_sin_sms_final.show(n=30)

+-----------------+----+--------------+
 nombre|edad|n_dias_sin_sms|
+-----------------+----+--------------+
 Celia Castro| 47| 0|
 David Roldan| 74| 1|
 Roberta Varado| 64| 1|
 Silvia Rodriguez| 49| 2|
 Marta Rodrigo| 67| 2|
 Roberto Salazar| 68| 3|
 Alvaro Monroy| 75| 3|
 Ana Robles| 41| 3|
Cristian Cuadrado| 52| 5|
 Laura Luiz| 44| 7|
 Victor Ruiz| 53| 7|
 Eduardo Redondo| 58| 7|
 Pablo Lopez| 40| 11|
 Ines Barcero| 29| 13|
 Pablo Perez| 26| 14|
 Carmen Arauzo| 19| 19|
 Melisa Aguado| 30| 20|
 Fernanda Gomez| 78| 21|
 Luis Rodrigues| 20| 21|
 Jorge Recio| 18| 22|
 Romina Verde| 29| 23|
 Clara Suarez| 21| 26|
+-----------------+----+--------------+

**9. Queremos obtener un Coeficiente de Ponderación que nos permita evaluar a cada cliente en función de su consumo para identificar los clientes más atractivos que forman parte de nuestra compañía. ESTE CÁLCULO SÓLO SE REALIZARÁ PARA LOS CLIENTES QUE TENGAN UNA SOLA OFERTA CONTRATADA CON LA COMPAÑÍA. </br> Este coeficiente se obtendrá en base a los consumos diarios, y por tanto sólo se tendrán en cuenta los clientes que existen en el mes de Agosto, ya que no tenemos datos de consumo del mes de Julio. </br> Las ponderaciones que se darán a cada uno de los consumos son las siguientes:** 
- 0.4 --> llamadas desde telefóno móvil.
- 0.3 --> datos_móviles_MB
- 0.2 --> llamadas desde teléfono fijo
- 0.1 --> SMS enviados </br> 

**Los pasos a seguir son los siguientes:**
  1. Obtener la suma de todos los días de cada uno de los 4 consumos para cada cliente.
  2. Obtener el máximo del cálculo anterior de todos los clientes para poder obtener un valor entre 0 y 1 para cada uno de los 4 consumos (recordar que sólo aplica para los clientes con UNA SOLA OFERTA CONTRATADA en el mes de Agosto. El cliente con mayor consumo de datos tendrá un valor de 1 en la columna 'datos_móviles_0_1')
  3. Multiplicar estas columnas obtenidas con valor entre 0 y 1 por su ponderación correspondiente (por ejemplo la columna "datos_moviles_0_1" se multiplicará por la ponderación de datos móviles 0.4).
  4. Por último se calculará la suma de las 4 ponderaciones obtenidas para calcular el Coeficiente de ponderación buscado (coeficiente_cliente), casteado a 3 decimales.
  5. Mostrar DF resultante ordenado por este coeficiente en sentido descendente, de manera que el primer registro corresponderá al cliente más atractivo. **El DF resultante tendrá 3 columnas: nombre, edad, coeficiente_cliente.**

In [0]:
# Necesitamos el df_consumos_diarios porque es de donde vamos a sacar los datos para calcular el coeficiente_cliente, el df_facturas_mes_actual porque los consumos corresponden al mes actual y necesitamos filtrar los clientes con una 
# oferta contratada, y el df_clientes para obtener el nombre y la edad del cliente.

# No necesitamos por tanto la tabla df_facturas_mes_ant ni df_ofertas

df_facturas_mes_actual = spark.read.table("df_facturas_mes_actual")
df_consumos_diarios = spark.read.table("df_consumos_diarios")
df_clientes = spark.read.table("df_clientes")

df_facturas_mes_actual.orderBy("id_cliente", "id_oferta").show(5)

+----------+---------+-------+----------+
id_cliente|id_oferta|importe| fecha|
+----------+---------+-------+----------+
 1| 6| 118.6|2020-08-31|
 1| 6| 124.5|2020-08-31|
 2| 4| 95.0|2020-08-31|
 3| 5| 102.5|2020-08-31|
 4| 1| 70.0|2020-08-31|
+----------+---------+-------+----------+
only showing top 5 rows

In [0]:
# Primero deberemos obtener los clientes que tienen una sola oferta en la compañía. Para ello, utilizando el df_facturas_mes_actual, agruparemos por id_cliente y calcularemos el comando countDistinct sobre id_oferta.
# Luego filtraremos la tabla por aquellos registros con valor 1 en la columna que acabamos de crear.

# Hemos utilizado CounDistinct sobre id_oferta porque el enunciado especifica clientes con una oferta (aunque pueden tener varios contratos con una misma oferta, como el id_cliente 1). Si quisieramos obtener los clientes con 1 solo contrato, despues de agrupar por cliente utilizaríamos el comando count sobre id_cliente, y filtraríamos igualmente por los registros cuyo valor en este columna es 1. 

df_clientes_1_oferta = df_facturas_mes_actual.groupBy("id_cliente").agg(countDistinct("id_oferta").alias("n_ofertas"))\
                                             .filter(col("n_ofertas") == 1)\
                                             .orderBy("id_cliente")


print("El número total de clientes con una oferta contratada es :", str(df_clientes_1_oferta.count()))
df_clientes_1_oferta.show()

El número total de clientes con una oferta contratada es : 20
+----------+---------+
id_cliente|n_ofertas|
+----------+---------+
 1| 1|
 2| 1|
 3| 1|
 4| 1|
 6| 1|
 7| 1|
 8| 1|
 10| 1|
 11| 1|
 12| 1|
 13| 1|
 14| 1|
 17| 1|
 19| 1|
 21| 1|
 23| 1|
 26| 1|
 27| 1|
 28| 1|
 29| 1|
+----------+---------+

In [0]:
# Ahora a los clientes con 1 oferta le agregamos los campos de la tabla de consumos, y eliminamos los que no necesitemos.

df_consumo_clientes_1_oferta = df_clientes_1_oferta.join(df_consumos_diarios, ["id_cliente"], "left").drop("n_ofertas","fecha")

df_consumo_clientes_1_oferta.show(10)

+----------+----------------+------------+----------------------+---------------------+
id_cliente|consumo_datos_MB|sms_enviados|minutos_llamadas_movil|minutos_llamadas_fijo|
+----------+----------------+------------+----------------------+---------------------+
 1| 1281| 2| 21| 0|
 1| 497| 3| 14| 0|
 1| 302| 0| 29| 0|
 1| 760| 0| 60| 12|
 1| 819| 1| 95| 18|
 1| 1729| 0| 11| 0|
 1| 734| 0| 19| 0|
 1| 760| 4| 56| 17|
 1| 830| 0| 32| 2|
 1| 1242| 1| 40| 0|
+----------+----------------+------------+----------------------+---------------------+
only showing top 10 rows

In [0]:
# Punto 1: Calculamos la suma de todos los días de cada uno de los 4 consumos para cada cliente.

df_consumo_total_clientes_1_oferta = df_consumo_clientes_1_oferta.groupBy("id_cliente").agg(sum("consumo_datos_MB").alias("total_consumo_datos_por_cliente"),
                                                                                            sum("sms_enviados").alias("total_sms_enviados_por_cliente"),
                                                                                            sum("minutos_llamadas_movil").alias("total_minutos_llamadas_movil_por_cliente"),
                                                                                            sum("minutos_llamadas_fijo").alias("total_minutos_llamadas_fijo_por_cliente"))

df_consumo_total_clientes_1_oferta.show(10)

+----------+-------------------------------+------------------------------+----------------------------------------+---------------------------------------+
id_cliente|total_consumo_datos_por_cliente|total_sms_enviados_por_cliente|total_minutos_llamadas_movil_por_cliente|total_minutos_llamadas_fijo_por_cliente|
+----------+-------------------------------+------------------------------+----------------------------------------+---------------------------------------+
 1| 29396| 38| 878| 283|
 2| 12605| 111| 1087| 759|
 3| 5474| 159| 689| 603|
 4| 21061| 27| 1630| 309|
 6| 3938| 275| 277| 1529|
 7| 35204| 21| 1115| 255|
 8| 9755| 170| 1921| 1036|
 10| 14653| 73| 531| 594|
 11| 39777| 15| 212| 144|
 12| 3602| 228| 491| 573|
+----------+-------------------------------+------------------------------+----------------------------------------+---------------------------------------+
only showing top 10 rows

In [0]:
# Punto 2: Ahora debemos obtener el máximo de cada columna calculada para contruir nuevas columnas que clasifiquen a los clientes con un valor entre 0 y 1 en función de cada consumo realizado, asignando un 1 a quien haya realizado el consumo máximo de cada tipo de consumo.

max_value1 = df_consumo_total_clientes_1_oferta.select(max(col('total_consumo_datos_por_cliente'))).collect()[0][0]
max_value2 = df_consumo_total_clientes_1_oferta.select(max(col('total_sms_enviados_por_cliente'))).collect()[0][0]
max_value3 = df_consumo_total_clientes_1_oferta.select(max(col('total_minutos_llamadas_movil_por_cliente'))).collect()[0][0]
max_value4 = df_consumo_total_clientes_1_oferta.select(max(col('total_minutos_llamadas_fijo_por_cliente'))).collect()[0][0]



df_consumo_total_clientes_1_oferta_0_1 =  df_consumo_total_clientes_1_oferta.withColumn("datos_moviles_0_1", (col("total_consumo_datos_por_cliente") / max_value1).cast(DecimalType(8,4)))\
                                                                                   .withColumn("sms_enviados_0_1", (col("total_sms_enviados_por_cliente") / max_value2).cast(DecimalType(8,4)))\
                                                                                   .withColumn("llamadas_movil_0_1", (col("total_minutos_llamadas_movil_por_cliente") / max_value3 ).cast(DecimalType(8,4)))\
                                                                                   .withColumn("llamadas_fijo_0_1", (col("total_minutos_llamadas_fijo_por_cliente") / max_value4 ).cast(DecimalType(8,4)))\
                                                                                   

df_consumo_total_clientes_1_oferta_0_1.show()

+----------+-------------------------------+------------------------------+----------------------------------------+---------------------------------------+-----------------+----------------+------------------+-----------------+
id_cliente|total_consumo_datos_por_cliente|total_sms_enviados_por_cliente|total_minutos_llamadas_movil_por_cliente|total_minutos_llamadas_fijo_por_cliente|datos_moviles_0_1|sms_enviados_0_1|llamadas_movil_0_1|llamadas_fijo_0_1|
+----------+-------------------------------+------------------------------+----------------------------------------+---------------------------------------+-----------------+----------------+------------------+-----------------+
 1| 29396| 38| 878| 283| 0.5937| 0.1297| 0.4571| 0.1851|
 2| 12605| 111| 1087| 759| 0.2546| 0.3788| 0.5659| 0.4964|
 3| 5474| 159| 689| 603| 0.1106| 0.5427| 0.3587| 0.3944|
 4| 21061| 27| 1630| 309| 0.4254| 0.0922| 0.8485| 0.2021|
 6| 3938| 275| 277| 1529| 0.0795| 0.9386| 0.1442| 1.0000|
 7| 35204| 21| 1115| 255| 0.7110| 0.0717| 0.5804| 0.1668|
 8| 9755| 170| 1921| 1036| 0.1970| 0.5802| 1.0000| 0.6776|
 10| 14653| 73| 531| 594| 0.2959| 0.2491| 0.2764| 0.3885|
 11| 39777| 15| 212| 144| 0.8034| 0.0512| 0.1104| 0.0942|
 12| 3602| 228| 491| 573| 0.0728| 0.7782| 0.2556| 0.3748|
 13| 19973| 103| 306| 365| 0.4034| 0.3515| 0.1593| 0.2387|
 14| 32341| 18| 1087| 942| 0.6532| 0.0614| 0.5659| 0.6161|
 17| 49512| 13| 155| 147| 1.0000| 0.0444| 0.0807| 0.0961|
 19| 34968| 45| 482| 441| 0.7063| 0.1536| 0.2509| 0.2884|
 21| 45384| 15| 1264| 1102| 0.9166| 0.0512| 0.6580| 0.7207|
 23| 4502| 15| 1021| 1263| 0.0909| 0.0512| 0.5315| 0.8260|
 26| 12915| 220| 1408| 1137| 0.2608| 0.7509| 0.7330| 0.7436|
 27| 6512| 233| 570| 1109| 0.1315| 0.7952| 0.2967| 0.7253|
 28| 22768| 293| 268| 260| 0.4598| 1.0000| 0.1395| 0.1700|
 29| 48922| 14| 1340| 1225| 0.9881| 0.0478| 0.6976| 0.8012|
+----------+-------------------------------+------------------------------+----------------------------------------+---------------------------------------+-----------------+----------------+------------------+-----------------+

In [0]:
# Punto 3: Multiplicar estas columnas obtenidas con valor entre 0 y 1 por su ponderación correspondiente (por ejemplo la columna "datos_moviles_0_1" se multiplicará por la ponderación de datos móviles).
# Para no añadir más columnas nuevas, sobreescribiremos las columnas 0_1 con su valor ponderado y eliminaremos los consumos totales.

df_consumo_clientes_1_oferta_ponderados = df_consumo_total_clientes_1_oferta_0_1.withColumn("datos_moviles_0_1", (col("datos_moviles_0_1") * 0.3).cast(DecimalType(5,4)))\
                                                                                      .withColumn("sms_enviados_0_1", (col("sms_enviados_0_1") * 0.1).cast(DecimalType(5,4)))\
                                                                                      .withColumn("llamadas_movil_0_1", (col("llamadas_movil_0_1") * 0.4).cast(DecimalType(5,4)))\
                                                                                      .withColumn("llamadas_fijo_0_1", (col("llamadas_fijo_0_1") * 0.2).cast(DecimalType(5,4)))\
                                                                                      .drop("total_consumo_datos_por_cliente", "total_consumo_sms_enviados_por_cliente", 
                                                                                            "total_consumo_minutos_llamadas_movil_por_cliente", "total_consumo_minutos_llamadas_fijo_por_cliente")

df_consumo_clientes_1_oferta_ponderados.show(10)

+----------+------------------------------+----------------------------------------+---------------------------------------+-----------------+----------------+------------------+-----------------+
id_cliente|total_sms_enviados_por_cliente|total_minutos_llamadas_movil_por_cliente|total_minutos_llamadas_fijo_por_cliente|datos_moviles_0_1|sms_enviados_0_1|llamadas_movil_0_1|llamadas_fijo_0_1|
+----------+------------------------------+----------------------------------------+---------------------------------------+-----------------+----------------+------------------+-----------------+
 1| 38| 878| 283| 0.1781| 0.0130| 0.1828| 0.0370|
 2| 111| 1087| 759| 0.0764| 0.0379| 0.2264| 0.0993|
 3| 159| 689| 603| 0.0332| 0.0543| 0.1435| 0.0789|
 4| 27| 1630| 309| 0.1276| 0.0092| 0.3394| 0.0404|
 6| 275| 277| 1529| 0.0239| 0.0939| 0.0577| 0.2000|
 7| 21| 1115| 255| 0.2133| 0.0072| 0.2322| 0.0334|
 8| 170| 1921| 1036| 0.0591| 0.0580| 0.4000| 0.1355|
 10| 73| 531| 594| 0.0888| 0.0249| 0.1106| 0.0777|
 11| 15| 212| 144| 0.2410| 0.0051| 0.0442| 0.0188|
 12| 228| 491| 573| 0.0218| 0.0778| 0.1022| 0.0750|
+----------+------------------------------+----------------------------------------+---------------------------------------+-----------------+----------------+------------------+-----------------+
only showing top 10 rows

In [0]:
# Punto 4: se calculará la suma de las 4 ponderaciones obtenidas para calcular el Coeficiente de ponderación buscado (coeficiente_cliente), casteado a 3 decimales. (En este caso casteamos a 3 enteros y 3 decimales)

df_consumo_clientes_1_oferta_ponderados_suma = df_consumo_clientes_1_oferta_ponderados.withColumn("coeficiente_cliente", 
                                                                                                  (col("datos_moviles_0_1") + 
                                                                                                   col("sms_enviados_0_1") + 
                                                                                                   col("llamadas_movil_0_1") + 
                                                                                                   col("llamadas_fijo_0_1")).cast(DecimalType(3,3)))

df_consumo_clientes_1_oferta_ponderados_suma.show(10)

+----------+------------------------------+----------------------------------------+---------------------------------------+-----------------+----------------+------------------+-----------------+-------------------+
id_cliente|total_sms_enviados_por_cliente|total_minutos_llamadas_movil_por_cliente|total_minutos_llamadas_fijo_por_cliente|datos_moviles_0_1|sms_enviados_0_1|llamadas_movil_0_1|llamadas_fijo_0_1|coeficiente_cliente|
+----------+------------------------------+----------------------------------------+---------------------------------------+-----------------+----------------+------------------+-----------------+-------------------+
 1| 38| 878| 283| 0.1781| 0.0130| 0.1828| 0.0370| 0.411|
 2| 111| 1087| 759| 0.0764| 0.0379| 0.2264| 0.0993| 0.440|
 3| 159| 689| 603| 0.0332| 0.0543| 0.1435| 0.0789| 0.310|
 4| 27| 1630| 309| 0.1276| 0.0092| 0.3394| 0.0404| 0.517|
 6| 275| 277| 1529| 0.0239| 0.0939| 0.0577| 0.2000| 0.376|
 7| 21| 1115| 255| 0.2133| 0.0072| 0.2322| 0.0334| 0.486|
 8| 170| 1921| 1036| 0.0591| 0.0580| 0.4000| 0.1355| 0.653|
 10| 73| 531| 594| 0.0888| 0.0249| 0.1106| 0.0777| 0.302|
 11| 15| 212| 144| 0.2410| 0.0051| 0.0442| 0.0188| 0.309|
 12| 228| 491| 573| 0.0218| 0.0778| 0.1022| 0.0750| 0.277|
+----------+------------------------------+----------------------------------------+---------------------------------------+-----------------+----------------+------------------+-----------------+-------------------+
only showing top 10 rows

In [0]:
# Punto 5: Mostrar DF resultante ordenado por este coeficiente en sentido descendente, de manera que el primer registro corresponderá al cliente más atractivo. El DF resultante tendrá 3 columnas: nombre, edad, coeficiente_cliente.
# Para añadir el nombre y la edad, como hemos hecho anteriormente, ejecutaremos un left join sobre id_cliente, dejando como tabla de la izquierda la que hemos construido.

df_coeficiente_cliente = df_consumo_clientes_1_oferta_ponderados_suma.join(df_clientes.select("id_cliente", "nombre", "edad"), ["id_cliente"], "left")\
                                                                     .select("nombre", "edad", "coeficiente_cliente")\
                                                                     .orderBy(desc("coeficiente_cliente"))

display(df_coeficiente_cliente)

nombre,edad,coeficiente_cliente
Romina Verde,29,0.74
Fatima Cuevas,29,0.687
Silvia Rodriguez,49,0.653
Celia Castro,47,0.595
Melisa Aguado,30,0.552
Pedro Conde,32,0.517
Carmen Arauzo,19,0.486
Eduardo Redondo,58,0.44
Pablo Perez,26,0.411
Fernanda Gomez,78,0.41


**10. Queremos averiguar la fecha en la que entre los 3 clientes que más datos consumen de cada grupo_edad llegan a un consumo de 20 GB de datos móviles. En caso de que algún grupo_edad no llegue entre los 3 clientes a 20 GB en todo el mes, se asignará "null" como valor de esta columna (Recordar que 1 GB = 1024 MB). </br> Obtener un DF que contiene 4 registros, uno para cada grupo_edad y 3 columnas: grupo_edad, fecha_20_GB, datos_moviles_total_grupo_3_clientes.** (datos_moviles_total_grupo_3_clientes representa el total de datos consumidos en MB por los 3 clientes del grupo hasta final de mes)

In [0]:
df_consumos_diarios = spark.read.table("df_consumos_diarios")
df_clientes = spark.read.table("df_clientes")


# Para este ejercicio sólo necesitamos las tabla df_consumos_diarios (para calcular el consumo de datos móviles solicitado) y df_clientes (para añadir la edad, y en base a ella, la columna "grupo_edad")

# Por tanto lo primero que haremos es añadir a la tabla de consumos la edad del cliente y calcular la columna grupo_edad como se ha visto en el ejercicio 4. 
# Una vez hecho esto eliminaremos todos los campos que no necesitamos.

df_consumos_diarios_datos_grupo_edad = df_consumos_diarios.join(df_clientes.select("id_cliente", "edad"), ["id_cliente"], "left")\
                                                                     .withColumn("grupo_edad", when(col("edad") < 26, lit(1))
                                                                                 .when(col("edad").between(26,40),lit(2))
                                                                                 .when(col("edad").between(41,65),lit(3))
                                                                                 .otherwise(lit(4)))\
                                                                     .drop("sms_enviados","minutos_llamadas_movil", "minutos_llamadas_fijo")


df_consumos_diarios_datos_grupo_edad.show()

+----------+----------------+----------+----+----------+
id_cliente|consumo_datos_MB| fecha|edad|grupo_edad|
+----------+----------------+----------+----+----------+
 27| 249|2020-08-16| 64| 3|
 27| 242|2020-08-17| 64| 3|
 27| 237|2020-08-18| 64| 3|
 27| 264|2020-08-19| 64| 3|
 27| 255|2020-08-20| 64| 3|
 27| 251|2020-08-21| 64| 3|
 27| 281|2020-08-22| 64| 3|
 27| 338|2020-08-23| 64| 3|
 27| 201|2020-08-24| 64| 3|
 27| 277|2020-08-25| 64| 3|
 27| 299|2020-08-26| 64| 3|
 27| 139|2020-08-27| 64| 3|
 27| 267|2020-08-28| 64| 3|
 27| 178|2020-08-29| 64| 3|
 27| 177|2020-08-30| 64| 3|
 27| 87|2020-08-31| 64| 3|
 28| 1124|2020-08-01| 54| 3|
 28| 1044|2020-08-02| 54| 3|
 28| 755|2020-08-03| 54| 3|
 28| 1533|2020-08-04| 54| 3|
+----------+----------------+----------+----+----------+
only showing top 20 rows

In [0]:
# Filtramos el df de consumos por grupo edad, agrupamos por id_cliente para calcular el total de datos consumido por cada cliente. Por último, ordenamos la tabla por el consumo total de datos recién calculado, para obtener los clientes que más datos han consumido.

df_consumos_diarios_datos_grupo_edad_1 = df_consumos_diarios_datos_grupo_edad.filter(col("grupo_edad") == 1)\
                                                                                 .groupBy("id_cliente").agg(sum("consumo_datos_MB").alias("consumo_total_datos"))\
                                                                                 .orderBy(desc("consumo_total_datos"))

df_consumos_diarios_datos_grupo_edad_2 = df_consumos_diarios_datos_grupo_edad.filter(col("grupo_edad") == 2)\
                                                                                 .groupBy("id_cliente").agg(sum("consumo_datos_MB").alias("consumo_total_datos"))\
                                                                                 .orderBy(desc("consumo_total_datos"))

df_consumos_diarios_datos_grupo_edad_3 = df_consumos_diarios_datos_grupo_edad.filter(col("grupo_edad") == 3)\
                                                                                 .groupBy("id_cliente").agg(sum("consumo_datos_MB").alias("consumo_total_datos"))\
                                                                                 .orderBy(desc("consumo_total_datos"))

df_consumos_diarios_datos_grupo_edad_4 = df_consumos_diarios_datos_grupo_edad.filter(col("grupo_edad") == 4)\
                                                                                 .groupBy("id_cliente").agg(sum("consumo_datos_MB").alias("consumo_total_datos"))\
                                                                                 .orderBy(desc("consumo_total_datos"))


df_consumos_diarios_datos_grupo_edad_1.show()
df_consumos_diarios_datos_grupo_edad_2.show()
df_consumos_diarios_datos_grupo_edad_3.show()
df_consumos_diarios_datos_grupo_edad_4.show()

+----------+-------------------+
id_cliente|consumo_total_datos|
+----------+-------------------+
 22| 50398|
 17| 49512|
 11| 39777|
 7| 35204|
 9| 32536|
+----------+-------------------+

+----------+-------------------+
id_cliente|consumo_total_datos|
+----------+-------------------+
 29| 48922|
 21| 45384|
 25| 41831|
 19| 34968|
 14| 32341|
 1| 29396|
 4| 21061|
 10| 14653|
+----------+-------------------+

+----------+-------------------+
id_cliente|consumo_total_datos|
+----------+-------------------+
 28| 22768|
 15| 20359|
 13| 19973|
 18| 18525|
 26| 12915|
 2| 12605|
 8| 9755|
 5| 8637|
 27| 6512|
+----------+-------------------+

+----------+-------------------+
id_cliente|consumo_total_datos|
+----------+-------------------+
 30| 9019|
 3| 5474|
 23| 4502|
 6| 3938|
 12| 3602|
+----------+-------------------+

In [0]:
# Como nos interesa obtener los 3 clientes con más consumo de cada grupo, utilizamos el comando collect()

first_consumer1 = df_consumos_diarios_datos_grupo_edad_1.collect()[0][0]
second_consumer1 = df_consumos_diarios_datos_grupo_edad_1.collect()[1][0]
third_consumer1 =  df_consumos_diarios_datos_grupo_edad_1.collect()[2][0]

first_consumer2 = df_consumos_diarios_datos_grupo_edad_2.collect()[0][0]
second_consumer2 = df_consumos_diarios_datos_grupo_edad_2.collect()[1][0]
third_consumer2 =  df_consumos_diarios_datos_grupo_edad_2.collect()[2][0]

first_consumer3 = df_consumos_diarios_datos_grupo_edad_3.collect()[0][0]
second_consumer3 = df_consumos_diarios_datos_grupo_edad_3.collect()[1][0]
third_consumer3 =  df_consumos_diarios_datos_grupo_edad_3.collect()[2][0]

first_consumer4 = df_consumos_diarios_datos_grupo_edad_4.collect()[0][0]
second_consumer4 = df_consumos_diarios_datos_grupo_edad_4.collect()[1][0]
third_consumer4 =  df_consumos_diarios_datos_grupo_edad_4.collect()[2][0]

print("Los id de los clientes que más consumen del grupo edad 1 son: " + str((first_consumer1, second_consumer1, third_consumer1)))
print("Los id de los clientes que más consumen del grupo edad 2 son: " + str((first_consumer2, second_consumer2, third_consumer2)))
print("Los id de los clientes que más consumen del grupo edad 3 son: " + str((first_consumer3, second_consumer3, third_consumer3)))
print("Los id de los clientes que más consumen del grupo edad 4 son: " + str((first_consumer4, second_consumer4, third_consumer4)))

Los id de los clientes que más consumen del grupo edad 1 son: (22, 17, 11)
Los id de los clientes que más consumen del grupo edad 2 son: (29, 21, 25)
Los id de los clientes que más consumen del grupo edad 3 son: (28, 15, 13)
Los id de los clientes que más consumen del grupo edad 4 son: (30, 3, 23)

In [0]:
# Ahora que ya sabemos que clientes poseen los mayores consumos de cada grupo_edad, necesitamos obtener la fecha en la que, la suma del consumo de estos tres clientes alcanza o supera los 20 gigas de datos móviles.
# Para ello, volveremos al df de consumos, filtraremos los registros por los id_clientes de estos clientes con mayor consumo, y lo ordenaremos por fecha de forma ascendente, para que nos aparezcan los consumos de los tres clientes en el mismo día  seguidos uno detras de otro, y siguiendo el orden natural de las fechas. 


df_consumos_diarios_grupo1_ordenado = df_consumos_diarios_datos_grupo_edad.filter((col("id_cliente") == first_consumer1 ) | (col("id_cliente") == second_consumer1) | (col("id_cliente") == third_consumer1 ))\
                                                                          .orderBy("fecha", "id_cliente")


df_consumos_diarios_grupo2_ordenado = df_consumos_diarios_datos_grupo_edad.filter((col("id_cliente") == first_consumer2 ) | (col("id_cliente") == second_consumer2) | (col("id_cliente") == third_consumer2 ))\
                                                                          .orderBy("fecha", "id_cliente")


df_consumos_diarios_grupo3_ordenado = df_consumos_diarios_datos_grupo_edad.filter((col("id_cliente") == first_consumer3 ) | (col("id_cliente") == second_consumer3) | (col("id_cliente") == third_consumer3 ))\
                                                                          .orderBy("fecha", "id_cliente")


df_consumos_diarios_grupo4_ordenado = df_consumos_diarios_datos_grupo_edad.filter((col("id_cliente") == first_consumer4 ) | (col("id_cliente") == second_consumer4) | (col("id_cliente") == third_consumer4 ))\
                                                                          .orderBy("fecha", "id_cliente")

df_consumos_diarios_grupo1_ordenado.show(10)
df_consumos_diarios_grupo2_ordenado.show(10)
df_consumos_diarios_grupo3_ordenado.show(10)
df_consumos_diarios_grupo4_ordenado.show(10)

+----------+----------------+----------+----+----------+
id_cliente|consumo_datos_MB| fecha|edad|grupo_edad|
+----------+----------------+----------+----+----------+
 11| 2134|2020-08-01| 20| 1|
 17| 1648|2020-08-01| 18| 1|
 22| 2403|2020-08-01| 21| 1|
 11| 1093|2020-08-02| 20| 1|
 17| 1849|2020-08-02| 18| 1|
 22| 2912|2020-08-02| 21| 1|
 11| 1467|2020-08-03| 20| 1|
 17| 1453|2020-08-03| 18| 1|
 22| 1098|2020-08-03| 21| 1|
 11| 1760|2020-08-04| 20| 1|
+----------+----------------+----------+----+----------+
only showing top 10 rows

+----------+----------------+----------+----+----------+
id_cliente|consumo_datos_MB| fecha|edad|grupo_edad|
+----------+----------------+----------+----+----------+
 21| 1483|2020-08-01| 29| 2|
 25| 1839|2020-08-01| 29| 2|
 29| 1524|2020-08-01| 29| 2|
 21| 1938|2020-08-02| 29| 2|
 25| 1555|2020-08-02| 29| 2|
 29| 987|2020-08-02| 29| 2|
 21| 1305|2020-08-03| 29| 2|
 25| 1412|2020-08-03| 29| 2|
 29| 1439|2020-08-03| 29| 2|
 21| 1324|2020-08-04| 29| 2|
+----------+----------------+----------+----+----------+
only showing top 10 rows

+----------+----------------+----------+----+----------+
id_cliente|consumo_datos_MB| fecha|edad|grupo_edad|
+----------+----------------+----------+----+----------+
 13| 530|2020-08-01| 53| 3|
 15| 539|2020-08-01| 52| 3|
 28| 1124|2020-08-01| 54| 3|
 13| 736|2020-08-02| 53| 3|
 15| 617|2020-08-02| 52| 3|
 28| 1044|2020-08-02| 54| 3|
 13| 649|2020-08-03| 53| 3|
 15| 729|2020-08-03| 52| 3|
 28| 755|2020-08-03| 54| 3|
 13| 698|2020-08-04| 53| 3|
+----------+----------------+----------+----+----------+
only showing top 10 rows

+----------+----------------+----------+----+----------+
id_cliente|consumo_datos_MB| fecha|edad|grupo_edad|
+----------+----------------+----------+----+----------+
 3| 123|2020-08-01| 68| 4|
 23| 222|2020-08-01| 78| 4|
 30| 329|2020-08-01| 67| 4|
 3| 460|2020-08-02| 68| 4|
 23| 62|2020-08-02| 78| 4|
 30| 222|2020-08-02| 67| 4|
 3| 291|2020-08-03| 68| 4|
 23| 26|2020-08-03| 78| 4|
 30| 173|2020-08-03| 67| 4|
 3| 149|2020-08-04| 68| 4|
+----------+----------------+----------+----+----------+
only showing top 10 rows

In [0]:
# Ahora que ya tenemos las tablas filtradas, y ordenadas por fecha, tenemos que calcular la suma acumulada de consumos que se va produciendo cada día, para lo cual vamos a aplicar una ventana sobre grupo_edad (porque queremos que sume todos los registros), pero esta vez utilizaremos el comando currentRow sobre la ventana, que mostrará la suma acumulada tras cada registro.
# Utilizaremos esta ventana también para añadir otra columna con el consumo total de estos 3 clientes, y de esta manera tendremos en estas tablas toda la información que necesitamos para la tabla final.

window_grupo_edad = Window.partitionBy("grupo_edad").orderBy("fecha", "id_cliente")


df_consumos_diarios_grupo1_ordenado = df_consumos_diarios_grupo1_ordenado.withColumn("consumo_diario_acumulado", sum("consumo_datos_MB")\
                                                                                     .over(window_grupo_edad.rowsBetween(Window.unboundedPreceding,Window.currentRow)))\
                                                                         .withColumn("datos_moviles_total_grupo_3_clientes", sum("consumo_datos_MB")\
                                                                                     .over(window_grupo_edad.rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing)))

df_consumos_diarios_grupo2_ordenado = df_consumos_diarios_grupo2_ordenado.withColumn("consumo_diario_acumulado", sum("consumo_datos_MB")\
                                                                                     .over(window_grupo_edad.rowsBetween(Window.unboundedPreceding,Window.currentRow)))\
                                                                         .withColumn("datos_moviles_total_grupo_3_clientes", sum("consumo_datos_MB")\
                                                                                     .over(window_grupo_edad.rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing)))

df_consumos_diarios_grupo3_ordenado = df_consumos_diarios_grupo3_ordenado.withColumn("consumo_diario_acumulado", sum("consumo_datos_MB")\
                                                                                     .over(window_grupo_edad.rowsBetween(Window.unboundedPreceding,Window.currentRow)))\
                                                                         .withColumn("datos_moviles_total_grupo_3_clientes", sum("consumo_datos_MB")\
                                                                                     .over(window_grupo_edad.rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing)))

df_consumos_diarios_grupo4_ordenado = df_consumos_diarios_grupo4_ordenado.withColumn("consumo_diario_acumulado", sum("consumo_datos_MB")\
                                                                                     .over(window_grupo_edad.rowsBetween(Window.unboundedPreceding,Window.currentRow)))\
                                                                         .withColumn("datos_moviles_total_grupo_3_clientes", sum("consumo_datos_MB")\
                                                                                     .over(window_grupo_edad.rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing)))

df_consumos_diarios_grupo1_ordenado.show(10)
df_consumos_diarios_grupo2_ordenado.show(10)
df_consumos_diarios_grupo3_ordenado.show(10)
df_consumos_diarios_grupo4_ordenado.show(10)

+----------+----------------+----------+----+----------+------------------------+------------------------------------+
id_cliente|consumo_datos_MB| fecha|edad|grupo_edad|consumo_diario_acumulado|datos_moviles_total_grupo_3_clientes|
+----------+----------------+----------+----+----------+------------------------+------------------------------------+
 11| 2134|2020-08-01| 20| 1| 2134| 139687|
 17| 1648|2020-08-01| 18| 1| 3782| 139687|
 22| 2403|2020-08-01| 21| 1| 6185| 139687|
 11| 1093|2020-08-02| 20| 1| 7278| 139687|
 17| 1849|2020-08-02| 18| 1| 9127| 139687|
 22| 2912|2020-08-02| 21| 1| 12039| 139687|
 11| 1467|2020-08-03| 20| 1| 13506| 139687|
 17| 1453|2020-08-03| 18| 1| 14959| 139687|
 22| 1098|2020-08-03| 21| 1| 16057| 139687|
 11| 1760|2020-08-04| 20| 1| 17817| 139687|
+----------+----------------+----------+----+----------+------------------------+------------------------------------+
only showing top 10 rows

+----------+----------------+----------+----+----------+------------------------+------------------------------------+
id_cliente|consumo_datos_MB| fecha|edad|grupo_edad|consumo_diario_acumulado|datos_moviles_total_grupo_3_clientes|
+----------+----------------+----------+----+----------+------------------------+------------------------------------+
 21| 1483|2020-08-01| 29| 2| 1483| 136137|
 25| 1839|2020-08-01| 29| 2| 3322| 136137|
 29| 1524|2020-08-01| 29| 2| 4846| 136137|
 21| 1938|2020-08-02| 29| 2| 6784| 136137|
 25| 1555|2020-08-02| 29| 2| 8339| 136137|
 29| 987|2020-08-02| 29| 2| 9326| 136137|
 21| 1305|2020-08-03| 29| 2| 10631| 136137|
 25| 1412|2020-08-03| 29| 2| 12043| 136137|
 29| 1439|2020-08-03| 29| 2| 13482| 136137|
 21| 1324|2020-08-04| 29| 2| 14806| 136137|
+----------+----------------+----------+----+----------+------------------------+------------------------------------+
only showing top 10 rows

+----------+----------------+----------+----+----------+------------------------+------------------------------------+
id_cliente|consumo_datos_MB| fecha|edad|grupo_edad|consumo_diario_acumulado|datos_moviles_total_grupo_3_clientes|
+----------+----------------+----------+----+----------+------------------------+------------------------------------+
 13| 530|2020-08-01| 53| 3| 530| 63100|
 15| 539|2020-08-01| 52| 3| 1069| 63100|
 28| 1124|2020-08-01| 54| 3| 2193| 63100|
 13| 736|2020-08-02| 53| 3| 2929| 63100|
 15| 617|2020-08-02| 52| 3| 3546| 63100|
 28| 1044|2020-08-02| 54| 3| 4590| 63100|
 13| 649|2020-08-03| 53| 3| 5239| 63100|
 15| 729|2020-08-03| 52| 3| 5968| 63100|
 28| 755|2020-08-03| 54| 3| 6723| 63100|
 13| 698|2020-08-04| 53| 3| 7421| 63100|
+----------+----------------+----------+----+----------+------------------------+------------------------------------+
only showing top 10 rows

+----------+----------------+----------+----+----------+------------------------+------------------------------------+
id_cliente|consumo_datos_MB| fecha|edad|grupo_edad|consumo_diario_acumulado|datos_moviles_total_grupo_3_clientes|
+----------+----------------+----------+----+----------+------------------------+------------------------------------+
 3| 123|2020-08-01| 68| 4| 123| 18995|
 23| 222|2020-08-01| 78| 4| 345| 18995|
 30| 329|2020-08-01| 67| 4| 674| 18995|
 3| 460|2020-08-02| 68| 4| 1134| 18995|
 23| 62|2020-08-02| 78| 4| 1196| 18995|
 30| 222|2020-08-02| 67| 4| 1418| 18995|
 3| 291|2020-08-03| 68| 4| 1709| 18995|
 23| 26|2020-08-03| 78| 4| 1735| 18995|
 30| 173|2020-08-03| 67| 4| 1908| 18995|
 3| 149|2020-08-04| 68| 4| 2057| 18995|
+----------+----------------+----------+----+----------+------------------------+------------------------------------+
only showing top 10 rows

In [0]:
# Ahora solamente tenemos que encontrar el primer registro de la columna consumo_diario_acumulado que supera 20 GB y obtenerlo para la tabla final
    
# Creamos una nueva columna llamada "fecha_20_GB" en la que informaremos la fecha de aquellos registros en los el consumo diario acumulado supera los 20 GB, de no cumplirse esta condición, la mantendremos vacía (None), con valores nulos.
  
_20_GB = (20 * 1024) 
print("20 GB equivalen a: " + str(_20_GB) + " MB")
    
df_consumos_diarios_grupo1_ordenado_fecha_20_GB = df_consumos_diarios_grupo1_ordenado.withColumn("fecha_20_GB", when(col("consumo_diario_acumulado") >= _20_GB, col("fecha")).otherwise(lit(None)))

df_consumos_diarios_grupo2_ordenado_fecha_20_GB = df_consumos_diarios_grupo2_ordenado.withColumn("fecha_20_GB", when(col("consumo_diario_acumulado") >= _20_GB, col("fecha")).otherwise(lit(None)))

df_consumos_diarios_grupo3_ordenado_fecha_20_GB = df_consumos_diarios_grupo3_ordenado.withColumn("fecha_20_GB", when(col("consumo_diario_acumulado") >= _20_GB, col("fecha")).otherwise(lit(None)))

df_consumos_diarios_grupo4_ordenado_fecha_20_GB = df_consumos_diarios_grupo4_ordenado.withColumn("fecha_20_GB", when(col("consumo_diario_acumulado") >= _20_GB, col("fecha")).otherwise(lit(None)))


                                                                                                 
df_consumos_diarios_grupo1_ordenado_fecha_20_GB.show()
df_consumos_diarios_grupo2_ordenado_fecha_20_GB.show()
df_consumos_diarios_grupo3_ordenado_fecha_20_GB.show()
df_consumos_diarios_grupo4_ordenado_fecha_20_GB.show()    

20 GB equivalen a: 20480 MB
+----------+----------------+----------+----+----------+------------------------+------------------------------------+-----------+
id_cliente|consumo_datos_MB| fecha|edad|grupo_edad|consumo_diario_acumulado|datos_moviles_total_grupo_3_clientes|fecha_20_GB|
+----------+----------------+----------+----+----------+------------------------+------------------------------------+-----------+
 11| 2134|2020-08-01| 20| 1| 2134| 139687| null|
 17| 1648|2020-08-01| 18| 1| 3782| 139687| null|
 22| 2403|2020-08-01| 21| 1| 6185| 139687| null|
 11| 1093|2020-08-02| 20| 1| 7278| 139687| null|
 17| 1849|2020-08-02| 18| 1| 9127| 139687| null|
 22| 2912|2020-08-02| 21| 1| 12039| 139687| null|
 11| 1467|2020-08-03| 20| 1| 13506| 139687| null|
 17| 1453|2020-08-03| 18| 1| 14959| 139687| null|
 22| 1098|2020-08-03| 21| 1| 16057| 139687| null|
 11| 1760|2020-08-04| 20| 1| 17817| 139687| null|
 17| 921|2020-08-04| 18| 1| 18738| 139687| null|
 22| 2183|2020-08-04| 21| 1| 20921| 139687| 2020-08-04|
 11| 1482|2020-08-05| 20| 1| 22403| 139687| 2020-08-05|
 17| 1623|2020-08-05| 18| 1| 24026| 139687| 2020-08-05|
 22| 2421|2020-08-05| 21| 1| 26447| 139687| 2020-08-05|
 11| 1004|2020-08-06| 20| 1| 27451| 139687| 2020-08-06|
 17| 1632|2020-08-06| 18| 1| 29083| 139687| 2020-08-06|
 22| 2182|2020-08-06| 21| 1| 31265| 139687| 2020-08-06|
 11| 2402|2020-08-07| 20| 1| 33667| 139687| 2020-08-07|
 17| 1829|2020-08-07| 18| 1| 35496| 139687| 2020-08-07|
+----------+----------------+----------+----+----------+------------------------+------------------------------------+-----------+
only showing top 20 rows

+----------+----------------+----------+----+----------+------------------------+------------------------------------+-----------+
id_cliente|consumo_datos_MB| fecha|edad|grupo_edad|consumo_diario_acumulado|datos_moviles_total_grupo_3_clientes|fecha_20_GB|
+----------+----------------+----------+----+----------+------------------------+------------------------------------+-----------+
 21| 1483|2020-08-01| 29| 2| 1483| 136137| null|
 25| 1839|2020-08-01| 29| 2| 3322| 136137| null|
 29| 1524|2020-08-01| 29| 2| 4846| 136137| null|
 21| 1938|2020-08-02| 29| 2| 6784| 136137| null|
 25| 1555|2020-08-02| 29| 2| 8339| 136137| null|
 29| 987|2020-08-02| 29| 2| 9326| 136137| null|
 21| 1305|2020-08-03| 29| 2| 10631| 136137| null|
 25| 1412|2020-08-03| 29| 2| 12043| 136137| null|
 29| 1439|2020-08-03| 29| 2| 13482| 136137| null|
 21| 1324|2020-08-04| 29| 2| 14806| 136137| null|
 25| 1242|2020-08-04| 29| 2| 16048| 136137| null|
 29| 1188|2020-08-04| 29| 2| 17236| 136137| null|
 21| 1837|2020-08-05| 29| 2| 19073| 136137| null|
 25| 1537|2020-08-05| 29| 2| 20610| 136137| 2020-08-05|
 29| 1730|2020-08-05| 29| 2| 22340| 136137| 2020-08-05|
 21| 2183|2020-08-06| 29| 2| 24523| 136137| 2020-08-06|
 25| 1492|2020-08-06| 29| 2| 26015| 136137| 2020-08-06|
 29| 1771|2020-08-06| 29| 2| 27786| 136137| 2020-08-06|
 21| 1735|2020-08-07| 29| 2| 29521| 136137| 2020-08-07|
 25| 1500|2020-08-07| 29| 2| 31021| 136137| 2020-08-07|
+----------+----------------+----------+----+----------+------------------------+------------------------------------+-----------+
only showing top 20 rows

+----------+----------------+----------+----+----------+------------------------+------------------------------------+-----------+
id_cliente|consumo_datos_MB| fecha|edad|grupo_edad|consumo_diario_acumulado|datos_moviles_total_grupo_3_clientes|fecha_20_GB|
+----------+----------------+----------+----+----------+------------------------+------------------------------------+-----------+
 13| 530|2020-08-01| 53| 3| 530| 63100| null|
 15| 539|2020-08-01| 52| 3| 1069| 63100| null|
 28| 1124|2020-08-01| 54| 3| 2193| 63100| null|
 13| 736|2020-08-02| 53| 3| 2929| 63100| null|
 15| 617|2020-08-02| 52| 3| 3546| 63100| null|
 28| 1044|2020-08-02| 54| 3| 4590| 63100| null|
 13| 649|2020-08-03| 53| 3| 5239| 63100| null|
 15| 729|2020-08-03| 52| 3| 5968| 63100| null|
 28| 755|2020-08-03| 54| 3| 6723| 

In [0]:
# Ahora filtraremos los registros que superan los 20GB de consumo diario acumulado. Como sólo nos interesa el primer registro que cumpla esta condición, una vez filtrado, utilizaremos la ventana que hemos definido anteriormente para aplicar el comando row_number() para que numere los registros y seleccionaremos el primero.
# Una vez hecho esto, seleccionamos las columnas que necesitamos para nuestra tabla final con el comando select()


window_grupo_edad = Window.partitionBy("grupo_edad").orderBy("fecha", "id_cliente")

grupo1filtrado = df_consumos_diarios_grupo1_ordenado_fecha_20_GB.filter(col("consumo_diario_acumulado") >= (20 * 1024))\
                                                                .withColumn("row_consumos>20GB", row_number().over(window_grupo_edad))\
                                                                .filter(col("row_consumos>20GB") == 1)\
                                                                .select("grupo_edad","fecha_20_GB", "datos_moviles_total_grupo_3_clientes")

grupo1filtrado.show()

+----------+-----------+------------------------------------+
grupo_edad|fecha_20_GB|datos_moviles_total_grupo_3_clientes|
+----------+-----------+------------------------------------+
 1| 2020-08-04| 139687|
+----------+-----------+------------------------------------+

In [0]:
# Ahora tenemos que repetir el mismo proceso para los grupos de edad restantes, y unir los dataframes finales.

grupo2filtrado = df_consumos_diarios_grupo2_ordenado_fecha_20_GB.filter(col("consumo_diario_acumulado") >= (20 * 1024))\
                                                                .withColumn("row_consumos>20GB", row_number().over(window_grupo_edad))\
                                                                .filter(col("row_consumos>20GB") == 1)\
                                                                .select("grupo_edad","fecha_20_GB", "datos_moviles_total_grupo_3_clientes")


grupo3filtrado = df_consumos_diarios_grupo3_ordenado_fecha_20_GB.filter(col("consumo_diario_acumulado") >= (20 * 1024))\
                                                                .withColumn("row_consumos>20GB", row_number().over(window_grupo_edad))\
                                                                .filter(col("row_consumos>20GB") == 1)\
                                                                .select("grupo_edad","fecha_20_GB", "datos_moviles_total_grupo_3_clientes")



grupo4filtrado = df_consumos_diarios_grupo4_ordenado_fecha_20_GB.filter(col("consumo_diario_acumulado") >= (20 * 1024))\
                                                                .withColumn("row_consumos>20GB", row_number().over(window_grupo_edad))\
                                                                .filter(col("row_consumos>20GB") == 1)\
                                                                .select("grupo_edad","fecha_20_GB", "datos_moviles_total_grupo_3_clientes")


grupo2filtrado.show()
grupo3filtrado.show()
grupo4filtrado.show()

+----------+-----------+------------------------------------+
grupo_edad|fecha_20_GB|datos_moviles_total_grupo_3_clientes|
+----------+-----------+------------------------------------+
 2| 2020-08-05| 136137|
+----------+-----------+------------------------------------+

+----------+-----------+------------------------------------+
grupo_edad|fecha_20_GB|datos_moviles_total_grupo_3_clientes|
+----------+-----------+------------------------------------+
 3| 2020-08-10| 63100|
+----------+-----------+------------------------------------+

+----------+-----------+------------------------------------+
grupo_edad|fecha_20_GB|datos_moviles_total_grupo_3_clientes|
+----------+-----------+------------------------------------+
+----------+-----------+------------------------------------+

In [0]:
# El dataframe del grupo 4 nos ha aparecido vacío, esto se debe a que no existe ningún registro que cumpla la condición de superar los 20 GB en el consumo diario acumulado. 
# Si revisamos este df, el consumo total que hemos calculado entre los tres clientes de este grupo es de 18995 MB, por debajo de los 20480 MB que estamos buscando. Por tanto, simplemente evitaremos filtrar el df por esta condición.

df_consumos_diarios_grupo4_ordenado_fecha_20_GB.show(1)


grupo4filtrado = df_consumos_diarios_grupo4_ordenado_fecha_20_GB.withColumn("row_consumos>20GB", row_number().over(window_grupo_edad))\
                                                                .filter(col("row_consumos>20GB") == 1)\
                                                                .select("grupo_edad","fecha_20_GB", "datos_moviles_total_grupo_3_clientes")

grupo4filtrado.show()

+----------+----------------+----------+----+----------+------------------------+------------------------------------+-----------+
id_cliente|consumo_datos_MB| fecha|edad|grupo_edad|consumo_diario_acumulado|datos_moviles_total_grupo_3_clientes|fecha_20_GB|
+----------+----------------+----------+----+----------+------------------------+------------------------------------+-----------+
 3| 123|2020-08-01| 68| 4| 123| 18995| null|
+----------+----------------+----------+----+----------+------------------------+------------------------------------+-----------+
only showing top 1 row

+----------+-----------+------------------------------------+
grupo_edad|fecha_20_GB|datos_moviles_total_grupo_3_clientes|
+----------+-----------+------------------------------------+
 4| null| 18995|
+----------+-----------+------------------------------------+

In [0]:
df_20GB_resumen = grupo1filtrado.unionByName(grupo2filtrado).unionByName(grupo3filtrado).unionByName(grupo4filtrado)

display(df_20GB_resumen)

grupo_edad,fecha_20_GB,datos_moviles_total_grupo_3_clientes
1,2020-08-04,139687
2,2020-08-05,136137
3,2020-08-10,63100
4,null,18995
